In [1]:
import sympy as sp
from src.core.sos_manager import SOS_Manager
from src.utils.text_process import PreprocessText as pl
from src.utils.text_process import poly_get_standard_form
from src.core.pqrsos.pqr_utils import pqr_ker, pqr_pqrt, pqr_cyc
from src.utils.basis_generator import generate_expr, deg
from src.core.sdpsos.sdpsos import SOS_SDP

a,b,c,x,y,z,w,u,v,r,s,t,p,q = sp.symbols('a b c x y z w u v r s t p q')
def monom_of(x, m):
    m = (m[0], m[1], sum(x.monoms()[0][-3:]) - m[0]-m[1])
    # print(x.monoms()[0])
    for i in range(m[0]):
        x = x.diff('a')
    x = x.subs('a',0)
    for i in range(m[1]):
        x = x.diff('b')
    x = x.subs('b',0)
    for i in range(m[2]):
        x = x.diff('c')
    x = x.subs('c',0)
    return x / sp.factorial(m[0]) / sp.factorial(m[1]) / sp.factorial(m[2])

def cubicdiscriminant(a,b,c,d):
    return b*b*c*c-4*a*c**3-4*b**3*d-27*a*a*d*d+18*a*b*c*d

def swa(x):
    a = str(x).replace(' ','').replace('**','^')
    print(a)
    return a

def sdesmos(x):
    a = (sp.latex(x).replace(' ',''))
    print(a)
    return a

def qinjiushao(f, x = None):
    if x is None: x = f.as_poly().gens
    f = f.as_poly(x[0])
    s = 0
    for c in f.all_coeffs():
        s *= x[0]
        s += qinjiushao(c, x[1:]) if len(x) > 1 else c
    return s

def hessian(f, value_a, value_b):
    a, b, c = sp.symbols('a b c')
    f = f.subs(c,1).as_poly(a,b)
    fa = f.diff(a)
    fb = f.diff(b)
    faa = fa.diff(a)
    fab = fa.diff(b)
    fbb = fb.diff(b)
    return (faa * fbb - fab**2).subs(a,value_a).subs(b,value_b)

## Quartic

取 $u = \frac{11-2\sqrt{105}}{13}$, $v =\frac{25-\sqrt{105}}{26}$, $C_1 =\frac{71-3\sqrt{105}}{288} $, $C_2 = \frac{\sqrt{105}- 1}{72}$, 则
$$\begin{aligned}&\quad \  \frac{567+45\sqrt{105}}{32\times 81}\left(\sum a\right)^4 - \sum a^3b 
\\ &= C_1\left[\sum a^2 - \frac{u^{2}-uv-u+v^{2}-v+3}{u+v-1}\sum  bc\right]^2 +
C_2\sum \left[a^2-b^2+u(ab-ac)+v(bc-ab)\right]^2
\end{aligned}
$$

In [39]:
poly = pl('ws(a)4-s(a3b)').as_expr().subs(w,(567+45*sp.sqrt(105))/32/81).as_poly(a,b,c)
# poly -= pl('zs(a2+wab)2').as_poly(a,b,c)
# z_ = sp.solve(poly.subs(a,1).subs(b,1).subs(c,1),z)[0]
# m_,p_,n_,q_ = [monom_of(poly,(4-i,i,0)) for i in range(4)]
# det = 3*m_*(m_+n_) - (p_**2+p_*q_+q_**2)
# # det.subs(z,z_).factor(extension = sp.sqrt(105))
# sp.simplify(m_.subs(z,z_).subs(w,(sp.sqrt(105)+5)/4))
# sdesmos(sp.together(sp.simplify((z_).subs(w,(sp.sqrt(105)+5)/4))))
C1 = (71-3*sp.sqrt(105))/288
C2 = (sp.sqrt(105) - 1)/72
u_, v_ = (11-2*sp.sqrt(105))/13, (25-sp.sqrt(105))/26
w_ = sp.simplify(((u**2 - u*v - u + v**2 - v + 3)/(u + v - 1)).subs(u,u_).subs(v,v_))
poly -= (C1 * pl('s(a2-wab)2').as_expr().subs(w,w_)).as_poly(a,b,c)
poly -= C2 * pl('s((a2-b2+u(ab-ac)+v(bc-ab))2)').as_expr().subs(u,u_).subs(v,v_).as_poly(a,b,c)
# poly # == 0

In [40]:
# ((u**2 - u*v - u + v**2 - v + 3)/(u + v - 1)).subs(u,0).subs(v,-1)
# # poly = pl('s((a2-b2+u(ab-ac)+v(bc-ab))2)+xs(a2-yab)2').subs(y,((u**2 - u*v - u + v**2 - v + 3)/(u + v - 1))).as_poly(a,b,c)
# sdesmos(pl('s(a4+6a3b-4ab3+8a2b2)-11abcs(a)').subs(c,1).subs(b,y).subs(a,x))

In [44]:
p,q,r,s,n = sp.symbols('p q r s n')
eq_coeffs = [
            -27*n + 9*p**2 + 9*p*q + 9*q**2 + 3*s - 27,
            81*n - 27*p**2 - 27*p*q + 6*p*s - 27*q**2 + 6*q*s - 3*s + 81,
            3*n*s - 81*n + 27*p**2 + 27*p*q - 6*p*s + 27*q**2 - 6*q*s + s**2 + 12*s - 81,
            -3*n*s + 27*n - 9*p**2 - 9*p*q - 9*q**2 + s**2 - 12*s + 27
        ]
eq = sum(eq_coeffs[i] * w ** (3 - i) for i in range(4)).as_poly(w)
det = sp.polys.discriminant(eq).factor().args[-1]
# sdesmos(eq.subs(p,-4).subs(q,-6).subs(n,11).subs(s,45).subs(w,x))
eqn = (sp.polys.discriminant((x**4+p*x**3+n*x**2+q*x+1).as_poly(x))).subs(n,x).as_poly(x)
# eqn.subs(n,0).factor()
r_ = sp.polys.nroots(eqn.subs(p,-10).subs(q,0), n = 60)[1]
# print(r_)

## Sextic

Positive numbers $abc=1$, $k\geqslant -\frac{189}{16}+\frac{9\sqrt{665}\cos{\left( {\operatorname{acos}{\left(-\frac{13117\sqrt{665}}{442225}\right)}}/{3}\right)}}{8}\approx 8.10864  $, then $\frac 1a+\frac 1b+\frac 1c+\frac{k}{a+b+c}\geqslant w$ where 

$w = \dfrac{3\cdot\sqrt[3]{2}\sqrt[3]{k^{2}+20k-8-\left(k-8\right)\sqrt{k^{2}-8k}}}{4}$

Change variables -> equivalent to `s(a2c)s(b2c)+ka2b2c2>=ws(a2c)abc`

In [ ]:
(a,b,c,x,y,z,k,w,t),p,q,r = sp.symbols('a b c x y z k w t'),sp.Rational(1),sp.sympify('(1-t*t)/3'),sp.symbols('r')
# (q/r + k/p)^3 >= w/r
p = (((q/r+k/p)**3-w/r)*r**3).expand()
cubicdiscriminant(*sp.collect(p, r).as_poly(r).all_coeffs()).factor()#.subs(k,9)#.subs(w,125)
# p.subs(w,9*k*k/4).factor()
# sdesmos(p.subs(k,9).subs(w,729/4).subs(t,sp.S(1)/4).subs(r,x))#.subs(r,sp.S(1)/8)
p.subs(t,sp.sqrt((1-4*w/9/k/k))).factor()
a,b,c = 0.20692550,0.45488669,1.#.39684369, .62995322, 1.
a,b,c = a/b,b/c,c/a
a,b,c = [i/(a+b+c) for i in (a,b,c)]
t0 = (1-3*(a*b+b*c+c*a))**.5
w0 = 5.95275**3#(1-4*5.9527**3/9/9/9)**.5
# w0 = 1/a+1/b+1/c+9/(a+b+c)
sdesmos(p.subs(t,t0).subs(w,w0).subs(k,9).subs(r,x))
# sdesmos((-4*r+q*q+18*q*r-4*q**3-27*r*r).subs(t,t0).subs(r,x))
(-4*r+q*q+18*q*r-4*q**3-27*r*r).factor()#.subs(t,sp.S(1)/4)
p2 = p.subs(r,(2*t**3-3*t*t+1)/27).factor()/(t-1)**3*19683
# cubicdiscriminant(*sp.collect(p2,k).as_poly(k).all_coeffs()).factor()
# sp.polys.nroots(p2.subs(t,t0).subs(k,15))[0]**(1/3)
k0 = 15
# sdesmos(((p2.subs(k,k0).subs(w,0) / p2.subs(k,k0).diff(w).subs(w,0))).subs(t,x))
# eq = (-p2.subs(w,0) / p2.diff(w).subs(w,0))
# swa(eq.diff(t).factor())
# t1 = sp.S(1)/4 + sp.S(3)/4*sp.sqrt(1-8/k) 
# t1 = sp.simplify(((k+sp.sqrt(k*k-8*k*(9-k)))/4/k))#.subs(k,15)
# eq.subs(t,t1).factor()
w = ((((k*k+20*k-8)-(k-8)*sp.sqrt(k*k-8*k))**(sp.S(1)/3)*3/2**(sp.S(5)/3)))
w.subs(k,20*sp.sin(sp.pi/10)+3).n(20)#(sp.sqrt(93)*434+3141)/9))#.n(20)

In [ ]:
pl('s(a4b2-a3b3)-(b2+c2-a2)a2(b-c)2-0b2(a-b)(a-c)(b2+c2-a2)').factor_list()
# 2a2b2-2a2bc+a2c2+ab3-abc2+b3c-b2c2
pl('2a2b2-2a2bc+a2c2+ab3-abc2+b3c-b2c2-(a2+b2-c2)c2').factor_list()
# 2a2b+ab2+abc+b2c-bc2-c3-(b-c)(2bc+2b2)-(b-c)(c2+a2-b2)
pl('s(a4b2-a3b3)-c2(a2+b2+c2-ab-bc-ca)(a2+b2-c2)').factor_list()
# # a3b+a3c-a2b2+a2c2-ab2c-2abc2+c4
# pl('s(a4b2-a3b3)-(c2(a2-ab-ac+bc+(b-c)2)(a2+b2-c2))').factor_list()
# 3a2b2-2a2bc+a2c2+ab3-abc2-b4+b3c-2b2c2
# pl('(2a2b2-2a2bc+a2c2+ab3-abc2+b3c-b2c2)a').subs('a','c').factor()
# pl('(2a2b2-2a2bc+a2c2+ab3-abc2+b3c-b2c2)-(2b-c)2a2').factor_list()
# (b2+c2-a2)a2(b-c)2+(a-b)(a-c)((2b-c)2a2+b(c-b)(2a2-ab-ac-bc))
pl('a3b+a3c-a2b2+a2c2-ab2c-2abc2+c4+abc2-(b2+c2-a2)c2').factor_list() # a2b+a2c+abc+2ac2+bc2
# s(a4b2-a3b3)-c2(a2+b2+c2-ab-bc-ca)(a2+b2-c2)+abc2(a-c)(b-c)-(a-c)(b-c)((b2+c2-a2)c2+(a-b)(a2b+a2c+abc+2ac2+bc2))
# (a-c)(b-c)(a-b)(a2b+a2c+abc+bc2)+abc2(a-c)(b-c)+c2(a2-ac-b2+bc)2
# (a-b)(a2b+a2c+abc+bc2)+abc2-0(b2+c2-a2)b2
(pl('(((2a2b2-2a2bc+a2c2+ab3-abc2+b3c-b2c2))-(2b-c)2(b-c)2)').factor_list()[-1][-1][0]).subs('a','c').factor()
# s(a4b2-a3b3)-(b2+c2-a2)a2(b-c)2-(a-b)(a-c)(2b-c)2(a+b-c)2
pl('((2a2b2-2a2bc+a2c2+ab3-abc2+b3c-b2c2))').factor_list()[-1][-1][0].subs('a','c').factor().subs('b','3/5*c')
# pl('s(a4b2-a3b3)-(b2+c2-a2)a2(b-c)2').subs('a','c').factor()#.subs('b','3/5*c')
# pl('s(a4b2-a3b3)-abc2(a-c)(b-c)-(a-b)(a3b2-a2c3-ab2c2+ac4-b3c2+b2c3)').factor_list()
pl('2a2b2-2a2bc+a2c2+ab3-abc2+b3c-b2c2').subs('a','c').factor().subs('b','3/5*c')
# (a3b2-a2c3-ab2c2+ac4-b3c2+b2c3-((b2+c2-a2)(a-b)2c+c(a2-ac-ab+bc)2+ab2(a-c)2))+0(a-c)
# pl('s(a4b2-a3b3)-((a2b2-2a2bc+a2c2+2(a-b)2(a-c)2)(b2+c2-a2))').as_expr().factor().args[-1].subs('a','c').factor()
a = sp.S(9)/8
sdesmos(pl('s(a4b2-a3b3)').subs('c',1).as_expr().subs('a',a).subs('b','x').factor())
print(sp.sqrt(a*a-1).n(20))
sp.collect(pl('s(a4b2-a3b3)').subs('c',1).as_expr(),'b').diff('b').factor() #.subs('b','sqrt(a*a-1)')
a,b,r = sp.symbols('a b r')
r = list(sp.polys.roots((4*b*b-3*b-3*a**3*b+2*a**4).as_poly(b)).keys())[1]
# sdesmos(pl('s(a4b2-a3b3)').subs('c',1).as_expr().subs(b,r).subs(a,'x').factor())
sdesmos(pl('s(a4b2-a3b3)').subs('c',1).as_expr().subs(b,sp.sqrt(a*a+1)).subs(a,'x').factor())

### Sextic Tree Real

In [245]:
eq = sp.groebner([u-x**3+3*x,v+3*x*(x-1)]).args[0][-1].subs({u: x, v: y})
# swa(eq.subs({x: u, y: v}))  # 27*x^2+27*x*y+54*x+y^3+18*y^2+54*y
x__ = sp.solve(eq.subs(y, (-6-v)/(2-u)*(x-2)-6).factor().args[-1].as_poly(x), x)[0].factor()
y__ = ((-6-v)/(2-u)*(x__-2)-6).factor()
t__ = sp.solve((t**3 - 3*t - x__).factor().args[1],t)[0]

https://tieba.baidu.com/p/8039371307 若 $x>0$, $3x+y+3\geqslant 0$, 且 $z = \dfrac{(y+3)^3}{27x}+x^2+xy-y+\dfrac{y^2}{3}$, 则对于非负实数 $a,b,c$ 有
$$\left(\sum \left(a^3b^3+xa^4bc-ya^3b^2c-ya^2b^3c\right)\right)+za^2b^2c^2\geqslant 0$$
证:(forever豪3) $pqr$ 法, 不妨 $p=a+b+c=1$, $q=ab+bc+ca$, $r=abc\geqslant 0$, 化简得
$$LHS=r^{2} \frac{\left(3 x + y + 3\right)^{3}}{27 x}+ r x - r q \left(3 x + y + 3\right) + q^3$$
若 $3x+y+3=0$ 则显然, 否则看成关于 $r$ 的二次函数, 求得判别式为
$$\Delta =- \frac{\left(3 q x + q y + 3 q - 3 x\right)^{2} \left(4(3x+y+3) q - 3 x\right)}{x}$$
当 $q\geqslant \dfrac{3x}{4(3x+y+3)}$, $\Delta\leqslant 0$, 二次项系数为正, 故 $LHS \geqslant 0$. 当 $q\leqslant \dfrac{3x}{4(3x+y+3)}$,一次项系数 $\geqslant \dfrac{x}{4}$ 为正, 也成立, 证毕.

<br>

https://tieba.baidu.com/p/8039371307 若 $x>0$, $3x+y+3\geqslant 0$, 且 $z = \dfrac{(y+3)^3}{27x}+x^2+xy-y+\dfrac{y^2}{3}$, 则对于非负实数 $a,b,c$ 有
$$\left(\sum \left(a^3b^3+xa^4bc-ya^3b^2c-ya^2b^3c\right)\right)+za^2b^2c^2\geqslant 0$$

 (星, forever豪3) 当 $x,y$ 使得如下 $w_1,w_2$ 非负时
$$\begin{aligned}w_1 &= - \frac{9 x^{2} + 6 x y - 306 x + y^{2} + 6 y + 9}{324 x \left(27 x^{3} + 27 x^{2} y + 81 x^{2} + 9 x y^{2} - 189 x y + 81 x + y^{3} + 9 y^{2} + 27 y + 27\right)}\\
w_2&=\frac{\left(9 x^{2} + 6 x y - 117 x + y^{2} + 15 y + 36\right)^{2}}{324 x \left(27 x^{3} + 27 x^{2} y + 81 x^{2} + 9 x y^{2} - 189 x y + 81 x + y^{3} + 9 y^{2} + 27 y + 27\right)}\\ \phi &=
\frac{36 x^{2} + 15 x y - 117 x + y^{2} + 6 y + 9}{9 x^{2} + 6 x y - 117 x + y^{2} + 15 y + 36}
\end{aligned}$$

则有配方
$$LHS\cdot  \sum a\sum ab = w_1\sum c\left[(3x+y-6)ab\left((3x+y+3)\sum ab-3x\left(\sum a\right)^2\right)-(6x-y-3)\left(3\left(\sum ab\right)^2-(3x+y+3)abc\sum a\right)\right]^2\\ +w_2\sum c\left[ab\left((3x+y+3)\sum ab-3x\left(\sum a\right)^2\right)-\phi\left(3\left(\sum ab\right)^2-(3x+y+3)abc\sum a\right)\right]^2 $$

In [ ]:
# monom_of(pl('s(a3b3+xa4bc-ya3b2c-ya2b3c)+za2b2c2-(s(ab)3+xabcs(a)3-(3x+y+3)abcs(a)s(ab)+(3x+3y+z+3)a2b2c2)'),(2,2,2))

x,y,z,p,q,r=sp.symbols('x'),sp.symbols('y'),sp.symbols('z'),sp.Rational(1),sp.sympify('(1-t*t)/3'),sp.symbols('r')
# q = sp.symbols('q')
poly = q**3+x*r*p**3-(3*x+y+3)*r*p*q+(3*x+3*y+z+3)*r*r 
poly2 = poly.diff('r').diff('r')/2
poly1 = poly.diff('r').subs('r',0)
poly0 = poly.subs('r',0)
det =  (poly1**2-poly0*poly2*4)*27
x,y = sp.sympify('3/2'),sp.sympify('13/4')
fz = lambda x,y:(y+3)**3/27/x+x*x+x*y-y+y*y/3
z = fz(x,y)
print('z =', z)
det.subs(zip('xyz',(sp.sympify('8/7'),sp.sympify('39/7'),sp.sympify('230/7')))).factor()
d = -4*r+q*q+18*q*r-4*q**3-27*r*r
m,n,p,q=sp.collect(det.factor(),'t').args[-1].args[1],sp.collect(det.factor(),'t').args[-2].args[1],sp.collect(det.factor(),'t').args[-3].args[1],det.subs('t',0)
det2 = (cubicdiscriminant(m,n,p,q).factor().args[-1]).subs(zip('xyz',(sp.sympify('3/2'),sp.sympify('6'),sp.sympify('141/4'))))
print(det2)
print(det.subs(zip('xyz',(sp.sympify('3/2'),sp.sympify('6'),sp.sympify('141/4')))).factor())
# (det2.subs('z',0) / -det2.diff('z').factor() - ( (y+3)**3/27/x)).factor()
# poly.subs('x',1).subs('y',6).subs('z',40).factor()
# d.subs(zip('xyz',(x,y,z))) 
# print(sp.latex(poly.subs('z',z)))
t2=(3*x+4*y+12)/(12*x+4*y+12)
print('Symme b=',(x-(1-t2)*(3*x+y+3)/3).factor())
print(d.subs('z',z).factor())
t = sp.symbols('t')
print(sp.latex(d.subs('z',z).factor()))#.subs('r',(-2*t**3-3*t*t+1)/27).factor()
poly.subs('x',x).subs('y',y).subs('z',z).args[-2].factor()

In [ ]:
x,y,z,q=sp.symbols('x'),sp.symbols('y'),sp.symbols('z'),sp.symbols('q')
fz = lambda x,y:(y+3)**3/27/x+x*x+x*y-y+y*y/3
z = fz(x,y)
# monom_of(pl('(s(a3b3+xa4bc-ya3b2c-ya2b3c)+za2b2c2)s(a)s(ab)'),(6,2,1))
# (3x+y+3)s(ab) - 3xs(a)2
# 3s(ab)2 - (3x+y+3)abcs(a)
q = 3*x/(3*x+y+3)
r=-(x-q*(3*x+y+3))/(3*x+y+3)**3/2*27*x
u=3*x+y+3 
# monom_of(pl('s(c((3x+y-6)((3x+y+3)s(ab)-3xs(a)2)ab-(6x-y-3)(3s(ab)2-(3x+y+3)abcs(a)))2)'),(6,2,1)).factor()#.subs(zip('abc',(1,1,1)))
# monom_of(pl('s(c(((3x+y+3)s(ab)-3xs(a)2)ab-w(3s(ab)2-(3x+y+3)abcs(a)))2)'),(5,4,0)).factor()
w = sp.symbols('w')
w2 = pl('(s(a3b3+xa4bc-ya3b2c-ya2b3c)+za2b2c2)s(a)s(ab)').subs(zip('abc',(1,1,1))).as_expr().factor() / pl('s(c(((3x+y+3)s(ab)-3xs(a)2)ab-w(3s(ab)2-(3x+y+3)abcs(a)))2)').subs(zip('abc',(1,1,1))).as_expr().factor()
w22 = ((9*(6*x-y-3)**2*x-9*x*x*(3*x+y-6)**2)/( 9*x*x*9*(6*x-y-3)**2-9*w*w*9*x*x*(3*x+y-6)**2)).factor()
weq = sp.fraction((w22 - w2).factor().subs('z',z))[0]
w3 = -(weq.subs('w',0) / weq.diff('w')).factor()

x,y=sp.sympify('2'),sp.sympify('8')
z = fz(x,y)
w3 = w3.subs('x',x).subs('y',y).subs('z',z)
# pl('(s(a3b3+xa4bc-ya3b2c-ya2b3c)+za2b2c2)s(a)s(ab)').subs(zip('abc',(1,1,1))).subs('z','230/7').subs('x','8/7').subs('y','39/7')
w2 = w2.subs('z',z).subs('w',w3).factor().subs('x',x).subs('y',y)
w1 = ((x - 9*x*x*w2)/(9*x*x*(3*x+y-6)**2)).factor().subs('x',x).subs('y',y)
# print((w1 - (1-9*w3*w3*w2)/81/(2*x-y-1)**2).factor())
print(f'(s(a3b3+{x}a4bc-{y}a3b2c-{y}a2b3c)+{z}a2b2c2)')
print(w1,w2,w3)
# assert False
# (w1*pl('s(c((3x+y-6)((3x+y+3)s(ab)-3xs(a)2)ab-(6x-y-3)(3s(ab)2-(3x+y+3)abcs(a)))2)').subs('x',x).subs('y',y).as_expr()+w2* pl('s(c(((3x+y+3)s(ab)-3xs(a)2)ab-w(3s(ab)2-(3x+y+3)abcs(a)))2)').subs('x',x).subs('y',y).subs('w',w3).as_expr()-pl('(s(a3b3+xa4bc-ya3b2c-ya2b3c)+za2b2c2)s(a)s(ab)').subs('x',x).subs('y',y).subs('z',z).as_expr().factor()).factor()
# a,b,c = sp.symbols('a b c')
# print(pl('(((3x+y+3)s(ab)-3xs(a)2)ab*u-v(3s(ab)2-(3x+y+3)abcs(a)))').as_poly(a,b,c).__repr__().replace(' ','').replace('(','{').replace(')','}').replace('x','x_').replace('y','y_').replace('**','^').replace('u','phi1').replace('v','phi2'))

### Hexagram

$$ \sum bc(a-b)(a-c)(a-ub)(a-uc)\geqslant 0$$

`s(a2bc(a2-b2+4(bc-ac)))+s(ac(3c+13/7b)(a-b)(3(a+b)c-4ab))`

In [ ]:
a,b,c,x,y,z,r,s,t,u,v,w = sp.symbols('a b c x y z r s t u v w')
poly = pl('s(a2bc(a2-b2+u(ab-ac)+v(bc-ab)))+s((ac(xc+yb)((uv-1)(a2c-b2c)-(u2+v)(a2b-abc)+(v2+u)(ab2-abc))))').as_poly(a,b,c)
# poly2 = poly#.subs(c,1)
# cyc = pqr_cyc(poly2.diff(a).diff(a)*poly2.diff(b).diff(b)-poly2.diff(a).diff(b)**2)#.as_expr().factor()
cyc1, cyc2 = [_.subs('p',1) for _ in pqr_cyc(poly)]
# (1 - 2q) / q = (u**2 - u*v - u + v**2 - v + 3)/(u + v - 1)
u_, v_ = 1, 2
q_ = (1/ ((u**2 - u*v - u + v**2 - v + 3)/(u + v - 1) + 2)).subs(u,u_).subs(v,v_)
r_ = sp.solve(pqr_ker().subs('p',1).subs('q',(1-t*t)/3).factor().args[1], r)[0]
detker = (cyc1**2 - cyc2**2*pqr_ker()).subs('p',1).subs(u,u).subs(v,v).subs(x,x).subs('q',(1-t*t)/3).subs(r,r_).factor().args[-1].args[0].as_poly(t)
# detker
detker2 = sp.polys.discriminant(detker.subs(u,u).subs(v,v).subs(x,x).as_poly(t)).factor().as_poly(x)
# q_ = (1/ ((u**2 - u*v - u + v**2 - v + 3)/(u + v - 1) + 2)).subs(u,2).subs(v,2)
# (cyc1**2 - cyc2**2*pqr_ker()).subs(u,3).subs(v,3).subs('p',1).subs(r,sp.S(1)/27).factor().args[-1].subs(x,sp.S(1)/2).subs(y,sp.S(14)/20).as_poly(sp.symbols('q')).factor_list()
# detker.as_expr().subs(u,1).subs(v,2).subs(x,1).subs(y,sp.S(3)/2).factor()
detker3 = sp.polys.discriminant(detker2).factor().args[-1]
u_, v_ = u, u
y_ = sp.solve(detker3, y)[0].factor().subs(u,u_).subs(v,v_)
x_ = sp.solve(detker2.subs(u,u_).subs(v,v_).subs(y,y_).factor().args[-1],x)[0]
print('x, y =', x_, y_)

### Hexagram Unsymmetric


In [224]:
# ker = pl('abcs((a2-b2+u(ab-ac)+v(bc-ab))2)+xs(c((uv-1)(a2c-b2c)-(u2+v)(a2b-abc)+(v2+u)(ab2-abc))2)').as_expr().subs(c,1).subs(b,-a-1).factor()
# ker = ker.args[-1]
# print(ker)
# ker = u**2*x - u*v*x + u*x + v**2*x + v*x + x - 2
# x_ = sp.solve(ker, x)[0]
# poly = pl('(u2-uv+v2+u+v+1)abcs((a2-b2+u(ab-ac)+v(bc-ab))2)+2s(c((uv-1)(a2c-b2c)-(u2+v)(a2b-abc)+(v2+u)(ab2-abc))2)').factor_list()[1][-1][0]
# print([monom_of(poly, i).factor() for i in [(3,3,0),(4,1,1),(3,2,1),(2,3,1),(2,2,2)]])
coeffs = [(u*v - 1)**2, u**2 - u*v + u + v**2 + v + 1, u**4 + u**3 - 3*u**2*v - 2*u*v**3 + 3*u*v**2 + 2*u - 2*v**3 - 3*v - 1, -2*u**3*v - 2*u**3 + 3*u**2*v - 3*u*v**2 - 3*u + v**4 + v**3 + 2*v - 1, -(u**4 - 2*u**3*v - u**3 + u**2*v**2 + u**2 - 2*u*v**3 - 3*u*v + v**4 - v**3 + v**2)]
coeffs = [_ / coeffs[0] for _ in coeffs]
u_, v_ = 4, sp.S(4)
print('s({0}a3b3+{1}a4bc+{2}a3b2c+{3}a2b3c+{4}a2b2c2)'.format(*map(lambda x: sp.simplify(x.subs(u,u_).subs(v,v_)), coeffs)))

s(1a3b3+1/9a4bc+-13/9a3b2c+-13/9a2b3c+16/9a2b2c2)


In [362]:
coeffs = [(u*v - 1)**2, u**2 - u*v + u + v**2 + v + 1, u**4 + u**3 - 3*u**2*v - 2*u*v**3 + 3*u*v**2 + 2*u - 2*v**3 - 3*v - 1, -2*u**3*v - 2*u**3 + 3*u**2*v - 3*u*v**2 - 3*u + v**4 + v**3 + 2*v - 1, -(u**4 - 2*u**3*v - u**3 + u**2*v**2 + u**2 - 2*u*v**3 - 3*u*v + v**4 - v**3 + v**2)]
coeffs = [_ / coeffs[0] for _ in coeffs]
poly = (pl('s(xa3b3+ya4bc+za3b2c+wa2b3c+ra2b2c2)').as_expr().subs(zip((x,y,z,w,r), coeffs)) / coeffs[1]).as_poly(a,b,c)

# print(((((coeffs[2]+coeffs[3]+3) / coeffs[1]).factor() - (r-1)**2)/(-3)).subs(r,u+v).factor())

### Hexagon


``s(1a3b3+13/25a4bc+1a3b2c+-126/25a2b3c+63/25a2b2c2)+s(4(a2b-abc)-3(ab2-abc))2``

``s(a4c2+18a3b2c-18a3bc2-a2b2c2)+s(1a3b3+1a4bc+5a3b2c+-37/4a2b3c+9/4a2b2c2)``

``s(a4c2+18a3b2c-18a3bc2-a2b2c2)s(a)-4/441s(a((-7/2)c(a2-b2+3(ab-ac)+3/2(bc-ab))+3(7/2(b2a-c2a)-21/2(b2c-abc)+21/4(bc2-abc)))2)``

In [46]:
# a_, b_ = 0.394340747417973, 0.560225096226878
# sp.nsimplify((a_**2*b_+b_**2+a_-3*a_*b_) / (b_**2*a_+a_**2+b_-3*a_*b_))
poly = pl('(12(s(b(b3+2c3-2a3+2ca2+2ab2-4c2a)(a-b)2)-5/12s(a3-2a2b-ab2+2abc)2))').subs(c,1)#.subs(a,1.4978386429261392188).subs(b,1.8970740694496457390).subs(c,1)
# poly.subs(b,sp.polys.nroots(sp.diff(poly))[-1]).n(20)
# print(sp.polys.nroots(sp.diff(poly))[2].n(20))
a_, b_ = sp.nsolve((poly.diff(a).as_expr(), poly.diff(b).as_expr()), (a,b), (1.49,1.89) )
# poly.subs(a,a_).subs(b,b_).n(2 0)
a_, b_

### Symmetric Hexagon with Real Numbers


$$\sum a^2(b^2-c^2)^2+(ux^2-2)\sum (a^3b^3-a^2b^2c^2) +(u-2)\sum abc(a^3-abc)+z\sum abc(a^2b+ab^2-2abc)\geqslant 0$$

<br>

若下式对于 $a,b,c\in\mathbb R$ 成立 $u,v\geqslant 0$,

$$u\prod (a-b)^2+\sum bc(a-b)(a-c)(a-xb)(a-xc)+z\sum a^2bc(b-c)^2\geqslant 0$$

则边缘要求 $u\geqslant \frac{\min\{1,x^2\}}{4}$. 根据对称轴 $c=1,a=b$ 非负 (因式分解后是关于 $a$ 的二次函数) 要求 $z(2x-z)\geqslant 0$

$z$ 取值在开区间内时, 可以由端点的配方式加权得到.

`s((a+b)2((a+b)2-(a+c)2)((a+b)2-(b+c)2))`

`s((a2-ab+b2)(a2-ab+b2-b2+bc-c2)(a2-ab+b2-c2+ca-a2))`


`8(s(a2(b2-c2)2)-3/8p(a-b)2)s(a2)-3/2s(a3b+a3c-2a2bc)2-1/2s(c2(a+b)2(a2-b2+2(b2-c2))2)-s(a2+3ab)2s(a2b2-a2bc)` 错解: `8(s(a2(b2-c2)2)-3/8p(a-b)2)s(a2+37/27ab)+1/9p(a-b)2s(a2-ab)-40/27s(a2+3ab)2s(a2b2-a2bc)-98/27p(a+b)2s(a2-ab)`

`s(57a4b2-14a4bc+57a4c2+111a3b3-61a3b2c-61a3bc2-89a2b2c2)s(a2+3/2ab)-128/3s(a2+17/8ab)2s(a2b2-a2bc)-27/2s(a2b+ab2+7/9abc)2s(a2-ab)-5/6s(a3b+a3c+a2b2-3a2bc)2`


In [541]:
poly = pl('up(a-b)2+s(bc(a-b)(a-c)(a-xb)(a-xc))+zs(a2bc(b-c)2)')
sym = poly.subs(c,1).subs(b,a).factor().args[-1].as_poly(a)
sp.polys.discriminant(sym).factor()
# sp.polys.discriminant(poly.subs(c,0).subs(b,1).as_expr().factor().args[-1].as_poly(a)).factor() # <= 0
# (4*u*u-(1-2*u)**2).factor() # >= 0
# hessian(poly.as_poly(a,b,c),0,0).factor()
# sp.polys.discriminant(poly.subs(c,1).as_expr().subs(b,r*a).as_poly(a).coeffs()[-1].factor().as_poly(r))

1 - 4*u

#### Normal Case

In [390]:
# Case 1. x = 1
# poly = pl('8(s(a2(b2-c2)2)-3/8p(a-b)2)s(a2+xab)-ys(c2(a+b)2(a2-b2+2(b2-c2))2)-s(a2+3ab)2s(a2b2-a2bc)-wp(a+b)2s(a2-ab)-us(a3b+a3c-2a2bc)2').as_poly(a,b,c)
# sdesmos(pl('8(s(a2(b2-c2)2)-3/8p(a-b)2)s(a2+ab)*50-s(a3b+a3c-2a2bc)2').subs(c,1).subs(a,x).subs(b,y))

# Case 2. x = 3/2
poly = pl('s(57a4b2-14a4bc+57a4c2+111a3b3-61a3b2c-61a3bc2-89a2b2c2)s(a2+xab)-ys(a2+17/8ab)2s(a2b2-a2bc)-ws(a2b+ab2+7/9abc)2s(a2-ab)-us(a3b+a3c+a2b2-3a2bc)2').as_poly(a,b,c)
print(sp.solve(monom_of(poly,_) for _ in [(6,2,0),(5,3,0),(6,1,1),(5,2,1)]))


{u: 5/6, w: 27/2, x: 3/2, y: 128/3}


In [38]:
# (pl('8(s(a2(b2-c2)2)-3/8p(a-b)2)').subs(c,1).subs(a,-1).subs(b,-1))
# pl('((a2c-ac2-a2b+ab2+x(bc2-b2c))2)').as_expr().factor()
p1 = pl('(s(xa2b+ab2-(x+1)abc)2+s(a2b+xab2-(x+1)abc)2)/(x2+1)').as_poly(a,b,c)
z_ = u**2+u*v+v**2-2
poly =  (pl('s(a2(b2-c2)2)+(u2-2)s(a3b3-a2b2c2)+(v2-2)s(a4bc-a2b2c2)-zabcs(a2b+ab2-2abc)')).as_expr().subs(z,z_)
u_ = +sp.sqrt(2)*(x+1)/sp.sqrt(x*x+1)
v_ = u_ 
poly = poly.subs(v,v_).subs(u,u_).as_poly(a,b,c)
# sp.simplify(p1 - poly)

In [307]:
# pl('a2c-ac2-(a2b-ab2)+x(bc2-b2c)').as_expr().factor()
poly = (pl('s(a2(b2-c2)2)+(u2-2)s(a3b3-a2b2c2)+(v2-2)s(a4bc-a2b2c2)-zabcs(a2b+ab2-2abc)')).as_poly(a,b,c)
sym = poly.subs(c,1).subs(b,a).factor().args[-1].as_poly(a)
u_, v_ = sp.sqrt(sp.S(1)/1), sp.sqrt(sp.S(4)/1) # gradient at (0,0,1) requires v \in [-2,2]
u_, v_ = sp.S(u_), sp.S(v_)
z_ = u_**2-u_*v_+v_**2-2
# z_ = -38
txt = f's(a2(b2-c2)2)+({u_**2-2})s(a3b3-a2b2c2)+({v_**2-2})s(a4bc-a2b2c2)-({z_})abcs(a2b+ab2-2abc)'
print(txt)
poly = pl(txt)
poly = pl('s((b-c)2(a2-2(ab+ac)+4bc)2)/21')
print('det =', sp.polys.discriminant(poly.subs(c,1).subs(b,a).factor().args[-1].as_poly(a)))

s(a2(b2-c2)2)+(-1)s(a3b3-a2b2c2)+(2)s(a4bc-a2b2c2)-(1)abcs(a2b+ab2-2abc)
det = 0


In [718]:
poly = pl('s(a4b2+a4c2+(u-2)a4bc+(ux2-2)a3b3+(-ux2+ux-u+2)a2b2c(a+b)+(ux2-2ux+u-2)a2b2c2)').as_poly(a,b,c)
# sym = poly.as_expr().subs(c,1).subs(b,a).factor()
ker = hessian(poly,-1/x,-1/x).factor().args[-1].as_poly(u) # <= 0
u_ = sp.solve(ker, u)[0].factor()

# [sp.solve(pl('s(a3b+a3c+ya2b2-(y+2)a2bc)2').subs(a,1).subs(b,1).subs(c,-x).as_expr(),y)[0].factor()]
bases = ['s(a2+yab)2s(a2b2-a2bc)', 's(a2b+ab2+zabc)2s(a2-ab)', 's(a3b+a3c+wa2b2-(w+2)a2bc)2']
params = {y: (x**2 + 2)/(2*x - 1), z:2*(x**2 - x + 1)/(3*x), w:2*(x - 1)}
bases = [pl(_).as_expr().subs(params).as_poly(a,b,c) for _ in bases]
eq = poly * pl('s(a2+vab)').as_poly(a,b,c) - y * bases[0] - z * bases[1] - w * bases[2]
sol = sp.solve((monom_of(eq,_) for _ in [(6,2,0),(5,3,0),(6,1,1),(5,2,1)]), (y,z,w,v))
sol = dict((_[0], _[1].subs(u,u_).factor()) for _ in sol.items())
# sol[u] = u_
# res = eq.subs(sol).as_poly(a,b,c)  # res == 0

#### Hard Case of Real Hexagon

`85/336p(a-b)2+s(bc(a-b)(a-c)(a+b)(a+c))-16/15s(a2bc(b-c)2)`

**命题** (forever豪3) 当 $\big\{y\in [-2,1], x\not\in [-1,2]\big\}$ 或 $\big\{y\leqslant x-1, x\in[-1,2]\big\}$, 取

$$\left\{\begin{aligned}C_1 &= 3x^{2} - x y^{2} - 7 x y - 4 x + 2 y^{2} + 5 y + 5
\\ C_2 &= 6 x^{2} - 2 x y^{2} - 14 x y - 8 x + 3 y^{3} + 13 y^{2} + 10 y - 2
\\ C_3 &= - x^{2} y^{2} - 4 x^{2} y + 2 x^{2} + 2 x y^{2} + 2 x y + 8 x - 6 y - 6
\\ C_4 &= - 3 x^{2} y^{2} + 6 x^{2} y - 6 x^{2} + 3 x y^{3} - 5 x y^{2} + 10 x y + 4 x + 3 y^{3} - 5 y^{2} - 8 y - 2
\end{aligned}\right.$$

则如下式子对于 $a,b,c\in\mathbb R$ 成立:

$$f(a,b,c) = C_1\sum_{cyc}a^2(b^2-c^2)^2+C_2\sum_{cyc}abc(a^3-abc)+C_3\sum_{cyc}(a^3b^3-a^2b^2c^2)+C_4\sum_{cyc}a^2bc(b-c)^2\geqslant 0$$

**证明** 当 $y=x-1$ 或 $y=1$ 或 $y=-2$ 属于退化情况可以因式分解， 平凡. 

其它时候取 $u = \frac{\left(x+1\right)\left(x-y-1\right)\left(xy-4x+4y+2\right)}{\left(2x-y\right)\left(y-1\right)\left(y+2\right)^{2}}$, $v = -\frac{\left(x-y-1\right)^{3}}{\left(2x-y\right)\left(y-1\right)\left(y+2\right)}$, $w = \frac{xy+2y+2}{2x-y}$, $z = \frac{xy-2}{-x+y+1}$, 
$$\left\{\begin{aligned}g_1(a,b,c) &=  \big[\sum_{cyc}(a^2+xab) \big]^2\sum_{cyc}a^2(b-c)^2
\\ g_2(a,b,c)&=  \big[\sum_{cyc}(a^2b+ab^2+yabc) \big]^2\sum_{cyc}(a-b)^2
\\ g_3(a,b,c) &=  \big[\sum_{cyc}(a^3b+a^3c+za^2b^2-(z+2)a^2bc) \big]^2
\end{aligned}\right.$$
则

$$\frac{u+v+1}{C_1}f(a,b,c)\sum_{cyc} (a^2+wab) = \frac12 g_1(a,b,c)+\frac12 ug_2(a,b,c)+vg_3(a,b,c)\geqslant 0$$

In [679]:
# sdesmos(pl('85/336p(a-b)2+s(bc(a-b)(a-c)(a+b)(a+c))-16/15s(a2bc(b-c)2)').subs(c,1).subs(a,x).subs(b,y))
# poly = pqr_cyc(pl('up(a-b)2+s(bc(a-b)(a-c)(a+b)(a+c))-vs(a2bc(b-c)2)').as_poly(a,b,c))[0].subs(p,1).subs(q,(1-t)/3).as_poly(r)
# det = (sp.polys.discriminant(sp.polys.discriminant(poly).as_poly(t)).factor()).args[-1].as_poly(u)
# v_ = (1+t*t)/(1-t*t)*8-8
# det.subs(v,v_).subs(t,sp.S(1)/3).factor()
poly = pl('85/336p(a-b)2+s(bc(a-b)(a-c)(a+b)(a+c))-16/15s(a2bc(b-c)2)').subs(c,1).as_expr()
sol = sp.nsolve((poly.diff(a),poly.diff(b)),(a,b), (0.1,-0.3))
sol = [sp.nsimplify(_) for _ in sol]
y_ = (sp.solve(pl('s(a2+yab)').subs(c,1).subs(a,sol[0]).subs(b,sol[1]),y))[0]
z_ = (sp.solve(pl('s(a2b+ab2+zabc)').subs(c,1).subs(a,sol[0]).subs(b,sol[1]),z))[0]
w_ = (sp.solve(pl('s(a3b+a3c+wa2b2-(w+2)a2bc)').subs(c,1).subs(a,sol[0]).subs(b,sol[1]),w))[0]

bases = ['s(a2+yab)2s(a2b2-a2bc)', 's(a2b+ab2+zabc)2s(a2-ab)', 's(a3b+a3c+wa2b2-(w+2)a2bc)2']
params = {y: y_, z: z_, w: w_}
bases = [pl(_).as_expr().subs(params).as_poly(a,b,c) for _ in bases]
poly = pl('85/336p(a-b)2+s(bc(a-b)(a-c)(a+b)(a+c))-16/15s(a2bc(b-c)2)')
eq = poly * pl('s(a2+vab)').as_poly(a,b,c) - y * bases[0] - z * bases[1] - w * bases[2]
sol = sp.solve((monom_of(eq,_) for _ in [(6,2,0),(5,3,0),(6,1,1),(5,2,1)]), (y,z,w,v))
# sol = dict((_[0], _[1].subs(u,u_).factor()) for _ in sol.items())
print(params, sol)
print(eq.subs(sol))

{y: 23/5, z: -94/75, w: 8/5} {y: 1/252, z: 25/112, w: 13/504, v: -3/5}
0


In [1012]:
poly = pl('s(a2+xab)2s(a2b2-a2bc)+us(a2b+ab2+yabc)2s(a2-ab)+vs(a3b+a3c+ga2b2-(g+2)a2bc)2').as_poly(a,b,c).as_expr().subs('g',(x*y-2)/(y+1-x)).as_poly(a,b,c)
# print(poly.subs(c,0).subs(b,1).as_expr().factor().args[-1].subs(a,-1).factor().args[-1])
# border = poly.subs(c,0).subs(b,1).as_expr().factor().args[-1].as_poly(a).monic()
# # poly_main = sp.collect(poly.as_expr().subs(c,1),a).as_poly(a)
# rem = poly.subs(c,1) % pl('s(a2+wab)').subs(c,1).as_poly(a,b)
# # print(rem.subs({x:y_,y:z_,v:sp.S(13)/2,u:sp.S(225)/4,w:-3/sp.S(5)})) # == 0

# eqs = [
#     rem.coeffs()[1].factor().args[-1] - rem.coeffs()[2].factor().args[-1],
#     rem.coeffs()[0].factor().args[-1],
#     (rem.coeffs()[6].factor().args[-1] - 7*rem.coeffs()[-2].factor().args[-1]).factor()
# ]

# eqnou1 = (eqs[0].as_poly(u) % eqs[1].as_poly(u)).as_expr().factor().args[-1]
# eqnou2 = (eqs[0].as_poly(u) % eqs[2].as_poly(u)).as_expr().factor().args[-1]
# eqfin = sp.fraction((eqnou1.as_poly(v) % eqnou2.as_poly(v)).as_expr().factor())[0]
# eqfin = 9*(-w + x)*(w - 1)*(w + 1)*(y - 1)*(x - y - 1)**2*(w**3 - 8*w**2 + 5*w + 8)*(-2*w*x + w*y + x*y + 2*y + 2)*(-4*w**3*y + 3*w**2*y**2 + 6*w**2*y + 5*w**2 - 6*w*y - 6*w - 3*y**2 + 2*y + 1)
# eqfin = (-2*w*x + w*y + x*y + 2*y + 2)
w__ = (x*y + 2*y + 2)/(2*x - y) # w__ = sp.solve(eqfin,w)[0]
# eqfinv = (2*x*y**2 + 2*x*y - 4*x - y**3 - y**2 + 2*y)*v + x**3 - 3*x**2*y - 3*x**2 + 3*x*y**2 + 6*x*y + 3*x - y**3 - 3*y**2 - 3*y - 1
v__ = -(x - y - 1)**3/((2*x - y)*(y - 1)*(y + 2))
# u__ = sp.solve(eqs[1].subs(w,w__).subs(v,v__).factor().args[-1].as_poly(u),u)[0].factor()
u__ = (x + 1)*(x - y - 1)*(x*y - 4*x + 4*y + 2)/((2*x - y)*(y - 1)*(y + 2)**2)
# print(poly.subs(u,u__).subs(v,v__).as_expr().factor())
# hexagon = poly.subs(u,u__).subs(v,v__).subs(c,1).as_poly(a,b).factor_list()
coeffs = [3*x**2 - x*y**2 - 7*x*y - 4*x + 2*y**2 + 5*y + 5,
 6*x**2 - 2*x*y**2 - 14*x*y - 8*x + 3*y**3 + 13*y**2 + 10*y - 2,
 -x**2*y**2 - 4*x**2*y + 2*x**2 + 2*x*y**2 + 2*x*y + 8*x - 6*y - 6,
 -3*x**2*y**2 + 6*x**2*y - 6*x**2 + 3*x*y**3 - 5*x*y**2 + 10*x*y + 4*x + 3*y**3 - 5*y**2 - 8*y - 2]
hexagon = pl('s(ga2(b2-c2)2+h(a4bc-a2b2c2)+u(a3b3-a2b2c2)+v(a2b+ab2-2abc)abc)').as_expr().subs(zip(('g','h',u,v),coeffs)).as_poly(a,b,c)


# range: (y=x-1 and -1<=x<=2) or (-2<=y<=1 and y<x-1)

In [1042]:
x__, y__ = 3, sp.S(2)/3
keys = {x: sp.S(x__), y: sp.S(y__)}
g__ = ((x*y-2)/(-x+y+1)).subs(keys)
print(poly_get_standard_form(hexagon.subs(keys).as_poly(a,b,c), is_cyc = True) + '' + f's(a2+{w__.subs(keys)}ab)*{((u__+v__+1)/coeffs[0]).subs(keys)}')
print(f'(s(a2+{x__}ab)2s(a2(b-c)2)/2+{u__.subs(keys)/2}s(a2b+ab2+{y__}abc)2s((a-b)2)+{v__.subs(keys)}s(a3b+a3c+{g__}a2b2-{g__+2}a2bc)2)')

s(80/9a4b2+32/3a4bc+80/9a4c2+32/3a3b3-32/3a3b2c-32/3a3bc2-160/9a2b2c2)s(a2+1ab)*27/64
(s(a2+3ab)2s(a2(b-c)2)/2+9/8s(a2b+ab2+2/3abc)2s((a-b)2)+1/2s(a3b+a3c+0a2b2-2a2bc)2)


### Sextic Symmetric

**命题** $u,t\in\mathbb R$, $u\neq 1$. 令 $\displaystyle {\rm Hex}_u =\sum bc(a-b)(a-c)[a-(u-1)b][a-(u-1)c]$

注: ${\rm Hex}_u=\dfrac 12\displaystyle \sum (ab+ac-bc)(b-c)^2(b+c-ua)^2+ u\displaystyle \prod (a-b)^2$

则对于 $a,b,c\geqslant 0$, 有:

$$\sum bc(b-c)^2(b+c-ua)^2 + \frac{(t-2)^2}{(u-1)^2} {\rm Hex}_{u}\geqslant 2t\prod (a-b)^2$$

**证明** (星, forever豪3) 取 $\phi = \frac{tu-u^{2}+2u-3}{u-1}$,

$$\left({\rm LHS}- {\rm RHS}\right)\sum_{cyc} a =\sum_{cyc}a(b-c)^2(a^2+b^2+c^2-tab-tac+\phi bc)^2+\frac{(t-2u)^2}{2(u-1)^2}\sum_{cyc}abc (b-c)^2(b+c-ua)^2$$

<br>

**命题2**

$$2\sum a(b+c)(b-c)^2(b+c-ua)^2\geqslant   \sum ab(a-b)^2(a+b-uc)^2\geqslant 4\prod (a-b)^2$$

`s(bc((a-b)(a+b-uc)+(a-c)(a+c-ub))2)`

In [ ]:
# (s(ab(a-b)4+4abca(a-b)(a-c)-abc(a2b+ab2-2abc)))
# -9s(c(a-b)2(a2-3ac+c2+b2-3bc+1/3ab)2)+((s(a2bc(a2-b2+4(bc-ac)))+s(ac(3c+13/7b)(a-b)(3(a+b)c-4ab)))+9(s(ab(a2-b2+4(bc-ac))2)-6p(a-b)2))s(a)
# poly1 = pl('(s(ab(a2-b2+x(bc-ac))2)-4p(a-b)2)s(a)')
# poly2 = pl('s(c(a-b)2((a+b-c)2-(x-1)ab)2)+2abcs((a2-b2+x(bc-ac))2)')
# (poly1 - poly2).as_poly(a,b,c).as_expr().factor()
poly1 = pl('s(ab(a2-b2+u(bc-ac))2)').as_poly(a,b,c)
poly2 = pl('s(a2bc(a2-b2+u(ab-ac)+u(bc-ab)))+s(ac((u-1)c+xb)(a-b)((u-1)(a+b)c-uab))').as_expr().subs(x,(u*u-u+1)/(2*u-1)).as_poly(a,b,c)
poly = poly1 + ((t-2)**2 / (u-1)**2 * poly2) - 2*t*pl('p(a-b)2').as_poly(a,b,c)
poly3 = poly * pl('s(a)').as_poly(a,b,c)
poly3 -= pl('s(a(b-c)2(a2+b2+c2-tab-tac+wbc)2)').as_expr().subs(w,(t*u-u*u+2*u-3)/(u-1)).as_poly(a,b,c)
poly3 -= (t-2*u)**2/(u-1)**2/2 * pl('abcs((a2-b2+u(bc-ac))2)').as_poly(a,b,c)
print(poly3.as_expr())
# ((t*u-u*u+2*u-3)/(u-1)).subs(t,8).subs(u,4)
t_, u_ = sp.S(5)/2, sp.S(400)/100
func = lambda x: monom_of(poly, x).subs(t,t_).subs(u,u_)
polyname = f's(ab(a4+b4)-{-func((4,2,0))}(a4b2+a2b4)+{func((3,3,0))}a3b3+{func((4,1,1))}a4bc+{func((3,2,1))}(a2b3c+a3b2c)+{func((2,2,2))/3}a2b2c2)'
print(polyname)

# print(pl('s(a)(2s(a(b+c)(b-c)2(b+c-ua)2)-s(ab(a-b)2(a+b-uc)2))') - pl('s(a(b-c)2(a2+b2+c2-2u(ab+ac)+(u+3)bc)2)'))

In [ ]:
# monom_of(poly, (4,1,1)).factor()
# list(sp.polys.roots((4*(q+2*r)-(p-r)**2 - 8).as_poly(r)).keys())[1]
# ((p-2)**2 - (p*p-q*4+8)).factor().subs(p,-4).subs(q,6)
det = pl('s(a5b+ab5+x(a4b2+a2b4)+ya3b3+wa4bc+z(a3b2c+a2b3c)-(2+2x+y+w+2z)a2b2c2)').subs(c,1).subs(b,a).as_expr().factor().args[-1].as_poly(a)
# sp.polys.discriminant(det).factor()
print(det.subs(x,p).subs(y,q).as_poly(a))

# s(ab(a4+b4)-6(a4b2+a2b4)+11a3b3+abca(a-b)(a-c)+(a3b2c+a2b3c)-3a2b2c2)-s(ab(a4+b4)-6(a4b2+a2b4)+11a3b3+350/363a4bc-647/4356(a2b3c+a3b2c)-3631/726/3a2b2c2)
# s(ab(a4+b4)-6(a4b2+a2b4)+11a3b3+13abca(a-b)(a-c)-3(a3b2c+a2b3c)+5a2b2c2)-s(ab(a4+b4)-6(a4b2+a2b4)+11a3b3+253/20a4bc+-1699/100(a2b3c+a3b2c)+2033/100a2b2c2)

In [ ]:
x,y,p,q,r,t=sp.symbols('x'),sp.symbols('y'),sp.Rational(1),sp.sympify('(1-t*t)/3'),sp.symbols('r'),sp.symbols('t')
poly = pqr_cyc(pl('s(a5b+ab5-2x(a4b2+a2b4)+(x2+2)a3b3-za4bc+w(a3b2c+a2b3c)+(4x-x2-4+z-2w)a2b2c2)').as_poly(a,b,c))[0]
poly = poly.subs('p',p).subs('q',q).as_poly(r)
sp.polys.discriminant(poly.subs(x,2).subs(z,4).subs(w,7).as_poly(r)).factor()
poly.subs(x,2).subs(z,4).subs(w,7).factor()#.subs(t,0).factor()

(forever豪3, 2023年1月1日) $x\in\mathbb R$, $a,b,c\geqslant 0$
$$\left\{\begin{aligned}
& \sum_{cyc}(a-b)^2\sum_{cyc}(a-b)^2(a+b-xc)^2\geqslant 3(x+1)^2\prod_{cyc}(a-b)^2
\\ & \sum_{cyc}ab(a-b)^2(a+b-xc)^2\geqslant 4\prod_{cyc}(a-b)^2
\end{aligned}\right.$$

In [ ]:
pl('s(a2-ab)s((a-b)2(a+b-3c)2)-24p(a-b)2').as_expr().factor()
# pl('s(a2-ab)s((a-b)2(a+b-4c)2)-38p(a-b)2').as_expr().factor()
print(pl('-2s(a3-abc-(x+1)/2(a2b+ab2-2abc))2+s(a2-ab)s((a-b)2(a+b-xc)2)').as_expr().factor())

### Sextic Full Symmetric

(forever豪3) 令 $x\in (-\infty,4]$, 

$$f(a,b,c) = a^6-(x+1)(a^5b+a^5c-a^3b^2c-a^3bc^2)+(4x-5)(a^4b^2+a^4c^2)-2(3x-5)a^3b^3-(2x+1)a^2b^2c^2$$

则对于 $a,b,c\geqslant 0$,

$$g(a,b,c)=\sum f(a,b,c)+ (x^2-4x+11)abc\sum a(a-b)(a-c)\geqslant 0$$

当 $x\in [1,4]$, 上式还对于 $a,b,c\in\mathbb R$ 均成立.


**证明** (星, forever豪3) 当 $x\in [1,4]$
$$g(a,b,c)\sum \left(a^2+(x-2)ab\right)
=\frac{(4-x)(x-1)}{2}\sum a^2(b-c)^2\left((a-b-c)^2-xbc\right)^2 + \sum (a^2-ab)\left((2x-x^2)abc-\prod (a+b-c)\right)^2$$

当 $x<1$,

$$\begin{aligned}g(a,b,c)\sum (a^2b^2-a^2bc) & =  \frac 12 abc\sum a(a-b)(a-c)\sum (b-c)^2\left(b+c-(x+1)a\right)^2
\\ & +\prod (a-b)^2 \left[\sum a^2(a-b)(a-c)+(2-x)\sum ab(a-b)^2+(6-6x)\sum a^2bc\right]\end{aligned}$$



``s((a-b-c)4a-abc(3a-b-c)2)s(a)-(s(ab(a2-b2+3(ab-ac)+3(bc-ab))2)-4p(a-b)2)``

``s(a2-2ab)2s(a2-ab)``

In [562]:
# pl('s(a3-abc-x(a2b+ab2-2abc))2').subs(a,sp.S(1)/2).subs(b,sp.S(1)/2).subs(c,1).as_expr().factor()
# poly.subs(c,0).as_expr().factor()
poly2 = pl('s(a4-x(a3b+ab3-2a2bc)+y(a2b2-a2bc)-a2bc)').as_expr().subs(c,1)
# sp.solvers.solve((poly2.subs(a,1).subs(b,0), poly2.subs(a,z).subs(b,z)), (x,y))
z_ = 7 / sp.S(2)
x_, y_ = (2*z_+1)/(2*z_), 1/z_
print(f's(a4-{x_}(a3b+ab3-2a2bc)+{y_}(a2b2-a2bc)-a2bc)')
# x_ = pl('s(a4-x(a3b+ab3-2a2bc)+y(a2b2-a2bc)-a2bc)2').as_expr().subs(c,1).subs(x,(2*z+1)/2/z).subs(y,1/z).as_poly(a,b,c) / pl('p(a-b)2s(a2-2ab)').subs(c,1)
# x_ = x_.factor()
# x_ = x_.subs(b, list(sp.polys.roots((a*a+b*b+1-a*b-a-b).as_poly(b)).keys())[1]).factor()
# x_ = 4 - 2 / z + 1/z/z/4
# poly = pl('s(a4-x(a3b+ab3-2a2bc)+y(a2b2-a2bc)-a2bc)2-up(a-b)2s(a2-2ab)').as_expr().subs(x,(2*z+1)/2/z).subs(y,1/z).subs(u,x_).as_poly(a,b,c)
# poly = poly.factor_list()[1][1][0]
z_ = sp.S(-2)/2
coeffs = [z*z, z*z+z, -5*z*z+4*z, 11*z*z-4*z+1, 10*z*z-6*z, -10*z*z+5*z-1, 10*z*z-6*z+1]
coeffs = [(_.subs(z,1/z) * z*z).factor() for _ in coeffs]
print('s({0}a6-{1}(a5b+a5c)+{2}(a4b2+a4c2)+{3}a4bc+{4}a3b3+{5}(a3b2c+a3bc2)+{6}a2b2c2)'.format(*map(lambda x: x.subs(z,z_), coeffs)))
# 's(a6-(x+1)(a5b+a5c)+(4x-5)(a4b2+a4c2)+(x2-4x+11)a4bc-2(3x-5)a3b3+(-x2+5x-10)(a3b2c+a3bc2)+(x2-6x+10)a2b2c2)'

s(a4-8/7(a3b+ab3-2a2bc)+2/7(a2b2-a2bc)-a2bc)
s(1a6-0(a5b+a5c)+-9(a4b2+a4c2)+16a4bc+16a3b3+-16(a3b2c+a3bc2)+17a2b2c2)


In [633]:
# CASE. A. x >= 1.
# poly = pl('s(a6-(x+1)(a5b+a5c)+(4x-5)(a4b2+a4c2)+(x2-4x+11)a4bc-2(3x-5)a3b3+(-x2+5x-10)(a3b2c+a3bc2)+(x2-6x+10)a2b2c2)s(a2+yab)-ws(a2(b-c)2((a-b-c)2-xbc)2)-s(a2+zab)((2x-x2)abc-p(a+b-c))2').as_poly(a,b,c)
# print(poly.subs(z,y-x+1).subs(w,-(x*y-2*x-y+2)/2).subs(y,x-2).as_poly(a,b,c))
# # w = (1-x)(x-4)/2
# # z = -1

# CASE. B. x < 1.
poly = -pl('s(a6-(x+1)(a5b+a5c)+(4x-5)(a4b2+a4c2)+(x2-4x+11)a4bc-2(3x-5)a3b3+(-x2+5x-10)(a3b2c+a3bc2)+(x2-6x+10)a2b2c2)s(a2b2-a2bc)-1/2abcs(a(a-b)(a-c))s((b-c)2(b+c-(x+1)a)2)').as_expr().factor().args[-1]
print((poly - pl('(6-6x)s(a2bc)+s(a2(a-b)(a-c))+(2-x)s(ab(a-b)2)').as_poly(a,b,c)))#.factor_list()
# swa(poly.subs(x,0))

Poly(0, a, b, c, domain='ZZ[x]')


## Sextic Symmetric Ultimate


$$\left[\sum \left(6a^3-11(a^2b+ab^2)+\frac{47}{3}abc\right)\right]^2\geqslant \prod [(2a-b)(2b-a)]$$

``4/9s(a3-abc-2(a2b+ab2-2abc))2+(s((a-b-c)4a -abc(3a-b-c)2)s(a)-(s(ab(a2-b2+3(ab-ac)+3(bc-ab))2)-4p(a-b)2))-7/3p(a-b)2``

``2s(a3-abc-2(a2b+ab2-2abc))2-13/4p(a-b)2+s(1a6-3(a5b+a5c)+3(a4b2+a4c2)+7a4bc+-2a3b3+-4(a3b2c+a3bc2)+2a2b2c2)``

``ABOVE =s(12a6-44a5b-44a5c-a4b2+262a4bc-a4c2+98a3b3-298a3b2c-298a3bc2+314a2b2c2)``

``s(35a6-148a5b-148a5c+233a4b2+440a4bc+233a4c2-168a3b3-460a3b2c-460a3bc2+443a2b2c2)-40p(a-b)2``

``s(414a6-1470a5b-1470a5c+979a4b2+5864a4bc+979a4c2+644a3b3-5584a3b2c-5584a3bc2+5228a2b2c2)``

``s(96a6-304a5b-304a5c+95a4b2+1266a4bc+95a4c2+290a3b3-1122a3b2c-1122a3bc2+1010a2b2c2)``

``s(a6-3(a5b+a5c)+(a4b2+a4c2)+3a3b3)+13abcs(a(a-b)(a-c))``

`s((b-c)2(7a2-b2-c2)2)/2-56p(a-b)2-4(s(bc(b-c)2(b+c-(1+2.44948974278)a)2)-8p(a-b)2+4s(bc(a-b)(a-c)(a-2.44948974278b)(a-2.44948974278c))/6)`

`s(a6-6a5b-6a5c+3a4b2+32a4bc+3a4c2+20a3b3-22a3b2c-22a3bc2-3a2b2c2)s(a2+7ab) -4s(ab(a-b)2(a2-4ab+b2-4c2-c(a+b))2)- s((a-b)2((a2-4ab+b2)(a+b)+c(a2+b2)+-19c2(a+b)+3abc+5c3)2)/2`


`https://tieba.baidu.com/p/8261574122`

In [109]:
poly = pl('s(a6-3a5b-3a5c+3a4b2+7a4bc+3a4c2-2a3b3-4a3b2c-4a3bc2+2a2b2c2)+xs(a3-abc-2(a2b+ab2-2abc))2-yp(a-b)2').subs({b:1,c:0}).as_poly(a)
ker = sp.polys.discriminant(poly).factor().args[-2].args[0]

**命题** (forever豪3) 参数 $u,v,m$,  定义
$$\left\{\begin{aligned}\Delta_1 &= -2m^{3}+\left(u+v+7\right)m^{2}-4\left(u+v+1\right)m+u^{2}v^{2}+4(u+v-1) \\ \Delta_2 &= 
4 m^{3} - \left(4u+4v+7\right) m^{2} + \left((u+v+4)^2+2uv-20\right) m + (uv-2)^2-2(u+v)^2\end{aligned}\right.$$

令 $w = \frac{m^3-(u+v+1)m^2+(uv+u+v-3)m + (uv-1)^2+2(u+v)+1}{m-2}$, $\phi =   -\frac{u^{2}v^{2}\Delta_2}{\left(m-2\right)\left(m+uv-2\right)^{2}\left(-2m+u+v-1\right)} - m $.

若 $u,v,m$ 满足条件: (1) ($\Delta_1=0$ 或 $\Delta_2=0$), 且 (2) $\phi\geqslant \max\left\{-1,\frac{\left(m-2\right)\left(u+v-2m\right)}{m+uv-2}\right\}$. 

则对于 $a,b,c\geqslant 0$:
$$f(a,b,c)= \sum_{cyc} (a-b)(a-c)(a-ub)(a-uc)(a-vb)(a-vc)- w\prod_{cyc}(a-b)^2\geqslant 0$$

**证** 取 $\lambda_1 = \frac{-m^{2}+\left(uv+1\right)m-uv(u+v+1)+2}{m+uv-2}$, $\lambda_2 = \frac{\left(1-2uv\right)m^{2}+\left(uv(u+v+1)-1\right)m+uv-2}{m+uv-2}$, $\lambda_3 = \frac{2\left(uv+u+v\right)m^2-\left(uv(u+v-1)+(u+v+2)^2-2\right)m+(uv-2)^2+2(u+v)^2}{m+uv-2}$, $\lambda_4 = \frac{\left(2uv-1\right)m-uv(u+v+1)+2}{m+uv-2}$.  

$$\left\{\begin{aligned}
 g(a,b,c) &= (a^2-mab+b^2)(a+b)+\lambda_1 c(a^2+b^2)+\lambda_2 c^2(a+b)+\lambda_3 abc+\lambda_4 c^3\\ 
h(a,b,c) & = a^2-mab+b^2+(uv + u + v - 1- m)c^2-(u+v-m)c(a+b)\end{aligned}\right.$$

则 

$$f(a,b,c)\sum_{cyc}(a^2+\phi ab) = \frac12\sum_{cyc} (a-b)^2g(a,b,c)^2 +\left[\phi  -\frac{\left(m-2\right)\left(u+v-2m\right)}{m+uv-2}\right] \sum_{cyc}ab(a-b)^2h(a,b,c)^2$$

In [3]:
# rem3 = pl('(s((a-b)(a-c)(a-gb)(a-gc)(a-hb)(a-hc))-wp(a-b)2)').subs({b:1,c:0}).as_poly(a).subs({'g':u, 'h':v}).as_poly(a) % (a*a-m*a+1).as_poly(a)
# w__ = sp.solve(rem3.as_expr().factor().args[-1].as_poly(w), w)[0].factor()
# ker = ((monom_of(poly,(7,1,0)) / monom_of(poly,(6,2,0)) - monom_of(comb2,(7,1,0)) / monom_of(comb2,(6,2,0)) ).factor()).args[-1].subs(w, w__).factor().args[-1]
# x__ = sp.solve(ker.as_poly(x), x)[0].factor()
w__ = (m**3 - m**2*u - m**2*v - m**2 + m*u*v + m*u + m*v - 3*m + u**2*v**2 - 2*u*v + 2*u + 2*v + 2)/(m - 2)
x__ = -(-2*m**5 - 4*m**4*u*v + m**4*u + m**4*v + 11*m**4 + 2*m**3*u**2*v**2 + 2*m**3*u**2*v + 2*m**3*u*v**2 + 14*m**3*u*v - 6*m**3*u - 6*m**3*v - 18*m**3 - 3*m**2*u**3*v**2 - 3*m**2*u**2*v**3 - 4*m**2*u**2*v**2 - 8*m**2*u**2*v - 8*m**2*u*v**2 - 8*m**2*u*v + 12*m**2*u + 12*m**2*v + 4*m**2 + m*u**4*v**2 + 4*m*u**3*v**3 + 6*m*u**3*v**2 + m*u**2*v**4 + 6*m*u**2*v**3 - 2*m*u**2*v**2 + 8*m*u**2*v + 8*m*u*v**2 - 8*m*u*v - 8*m*u - 8*m*v + 8*m + u**4*v**4 - 2*u**4*v**2 - 8*u**3*v**3 - 2*u**2*v**4 + 4*u**2*v**2)/((m - 2)*(m + u*v - 2)**2*(-2*m + u + v - 1))
# poly.subs({x: x__, w: w__}).as_poly(a,b,c).as_expr().factor()
# ratio = (monom_of(poly,(7,1,0)) / monom_of(comb2,(7,1,0))).factor().subs({x: x__, w: w__}).factor()
ratio = -(2*m**5 - 3*m**4*u - 3*m**4*v - 11*m**4 + 2*m**3*u**2*v**2 - 2*m**3*u**2*v + m**3*u**2 - 2*m**3*u*v**2 + 2*m**3*u*v + 17*m**3*u + m**3*v**2 + 17*m**3*v + 18*m**3 - 3*m**2*u**3*v**2 + m**2*u**3*v - 3*m**2*u**2*v**3 - 2*m**2*u**2*v**2 + 7*m**2*u**2*v - 6*m**2*u**2 + m**2*u*v**3 + 7*m**2*u*v**2 - 12*m**2*u*v - 30*m**2*u - 6*m**2*v**2 - 30*m**2*v - 4*m**2 + m*u**4*v**2 + 4*m*u**3*v**3 + 6*m*u**3*v**2 - 4*m*u**3*v + m*u**2*v**4 + 6*m*u**2*v**3 - 10*m*u**2*v**2 - 4*m*u**2*v + 12*m*u**2 - 4*m*u*v**3 - 4*m*u*v**2 + 24*m*u*v + 12*m*u + 12*m*v**2 + 12*m*v - 8*m + u**4*v**4 - 2*u**4*v**2 - 8*u**3*v**3 + 4*u**3*v - 2*u**2*v**4 + 12*u**2*v**2 - 4*u**2*v - 8*u**2 + 4*u*v**3 - 4*u*v**2 - 16*u*v + 8*u - 8*v**2 + 8*v)/((m - 2)*(m + u*v - 2)**2*(-2*m + u + v - 1))
# rem4 = pl('(s((a-b)(a-c)(a-gb)(a-gc)(a-hb)(a-hc))-wp(a-b)2)').subs({b:1,c:0}).as_poly(a).subs({'g':u, 'h':v}).subs({x: x__, w: w__}).as_poly(a) % (a*a-m*a+1).as_poly(a)**2
# dbkr = rem4.as_expr().factor().args[-1]
dbkr = -2*m**3 + m**2*u + m**2*v + 7*m**2 - 4*m*u - 4*m*v - 4*m + u**2*v**2 + 4*u + 4*v - 4
dbkr2 = 4*m**3 - 4*m**2*u - 4*m**2*v - 7*m**2 + m*u**2 + 4*m*u*v + 8*m*u + m*v**2 + 8*m*v - 4*m + u**2*v**2 - 2*u**2 - 8*u*v - 2*v**2 + 4

In [170]:
from tqdm import tqdm
mmms = [(7,1,0),(6,2,0),(5,3,0),(4,4,0),(6,1,1),(5,2,1),(4,3,1),(4,2,2),(3,3,2)]
for mmm in tqdm(mmms):
    fac = (monom_of(poly, mmm) - ratio * monom_of(comb2, mmm)).subs({x: x__, w: w__}).factor()
    if fac == 0:
        continue
    if dbkr not in fac.args:
        print('No dbkr  in', mmm)
    if dbkr2 not in fac.args:
        print('No dbkr2 in', mmm)

100%|██████████| 9/9 [00:46<00:00,  5.14s/it]


In [35]:
# u_, v_ = sp.polys.roots(pl('s((b2+c2-xa(b+c))2(b-c)2)-((x+1)2/2 + 4)p(a-b)2').as_poly(a,b,c).subs({b:a,c:1}).factor().args[-1].args[0].subs(a,1/a).factor().args[-1].as_poly(a))
# dbkr.subs({u: u_, v: v_}).factor()
# x__.subs({u: u_, v: v_, m: (x+3)/2}).factor()
# u_, v_ = sp.polys.roots(a*a-4*a-3)
# x__.subs({u:u_, v:v_, m:sp.S(7)/2}).factor()

In [34]:
# cases = {u:(5-sp.sqrt(41))/2,v:(5+sp.sqrt(41))/2, m:4}
# cases = {u: - sp.sqrt(6), v: sp.sqrt(6), m:4}
u_, v_ = sp.polys.roots(a*a-4*a-3)
cases = {u: u_, v: v_}
cases[m] = list(filter(lambda x: isinstance(x, sp.Rational), sp.polys.roots(dbkr.subs(cases))))[0]
params1_, params2_ = dict(list(cases.items())), dict(list(cases.items()))
for key, value in params.items():
    params1_[key] = sp.simplify(value.subs(cases))
print('s((a-b)2(%s)2)/2'%poly_get_standard_form(pl('(a2-mab+b2)(a+b)+xc(a2+b2)+yc2(a+b)+wabc+zc3').subs(params1_).as_poly(a,b,c)), end = '')
for key, value in params2.items():
    params2_[key] = sp.simplify(value.subs(cases))
ratio_ = sp.simplify(ratio.subs(cases))

print('+%ss(ab(a-b)2(%s)2)'%(ratio_, poly_get_standard_form(pl('a2-mab+b2+xc2-yc(b+a)').subs(params2_).as_poly(a,b,c))))
# print('multiplier', sp.simplify(x__.subs(cases)))
# print('ratio', sp.simplify(ratio.subs(cases)))

s((a-b)2(a3-5/2a2b+3/2a2c-5/2ab2+4abc-33/2ac2+b3+3/2b2c-33/2bc2+5c3)2)/2+4s(ab(a-b)2(a2-7/2ab-1/2ac+b2-1/2bc-7/2c2)2)


In [717]:
poly = pqr_cyc(pl('zs(a3-abc-2(a2b+ab2-2abc))2+(s((a-b-c)4a -abc(3a-b-c)2)s(a)-(s(ab(a2-b2+3(ab-ac)+3(bc-ab))2)-4p(a-b)2))-yp(a-b)2').as_poly(a,b,c))[0]
poly = poly.subs(p,1).subs(q,(1-t)/3).as_poly(r)
r2, r1, r0 = poly.all_coeffs()
det = (r1**2-4*r2*r0).factor()
detker = (sp.polys.discriminant(det.as_poly(t))).factor().args[-1]
# z = (4*(t - 4)**2/(9*(t - 2))),   y = (-t*t+16)/3
# print(sp.polys.discriminant(detker.as_poly(z)).factor()) # -64(3y-16)^3
# [_.factor() for _ in sp.polys.roots(detker.subs(y, (-t*t+16)/3).factor().as_poly(x))]

In [68]:
# poly = pqr_cyc(pl('zs(a3-abc-(x+2)/2(a2b+ab2-2abc))2+s(a6-(x+1)(a5b+a5c)+(4x-5)(a4b2+a4c2)+(x2-4x+11)a4bc-2(3x-5)a3b3+(-x2+5x-10)(a3b2c+a3bc2)+(x2-6x+10)a2b2c2)-yp(a-b)2').as_poly(a,b,c))[0]
# poly = poly.subs(p,1).subs(q,(1-t)/3).as_poly(r)
# r2, r1, r0 = poly.all_coeffs()
# det = (r1**2-4*r2*r0).factor()
# detker = (sp.polys.discriminant(det.as_poly(t))).factor()#.args[-1]
detker = (x - 1)**2*(x**4 - 12*x**3 + 4*x**2*y + 48*x**2 - 8*x*y - 64*x + 4*y)**2*(9*x**2*z + 4*x**2 + 36*x*z - 32*x + 12*y + 36*z + 64)*(729*x**4*z**3 + 864*x**4*z**2 + 256*x**4*z - 4374*x**3*z**3 - 4752*x**3*z**2 - 3328*x**3*z + 108*x**2*y*z**2 + 576*x**2*y*z + 9477*x**2*z**3 + 11988*x**2*z**2 + 15360*x**2*z - 216*x*y*z**2 - 2016*x*y*z - 8748*x*z**3 - 19008*x*z**2 - 28672*x*z + 64*y**2 + 108*y*z**2 + 2304*y*z + 2916*z**3 + 13824*z**2 + 16384*z)/81
detker = detker.args[-1].as_poly(y)
# detkerz = (sp.polys.discriminant(detker) / 2985984 / (32*x**3 - 252*x**2 + 3*x*y + 624*x - 3*y - 512)**2).factor()
# detkery = sp.polys.discriminant(detker).factor()
detkery = z*(x - 1)*(9*x*z - 16*x - 9*z + 64)  # detkery must be square
z_ = 16*t**2*(x - 4)*(x - 1)/((3*t*x - 3*t - 1)*(3*t*x - 3*t + 1))
# detker = detker.subs(z,z_).factor()
yroots = [-4*t*(x - 4)*(x - 1)*(72*t**2*x**3 - 351*t**2*x**2 + 567*t**2*x - 288*t**2 - 24*t*x**2 + 93*t*x - 96*t + 2*x - 8)/((3*t*x - 3*t - 1)**2*(3*t*x - 3*t + 1)),
 -4*t*(x - 4)*(x - 1)*(72*t**2*x**3 - 351*t**2*x**2 + 567*t**2*x - 288*t**2 + 24*t*x**2 - 93*t*x + 96*t + 2*x - 8)/((3*t*x - 3*t - 1)*(3*t*x - 3*t + 1)**2)]

x_ = sp.S(2)
t_ = sp.S(1)/7   # t < 1/3/(x-1) should be very small
y_ = yroots[1].subs(x,x_).subs(t,t_)
coeffs = [z*z, z*z+z, -5*z*z+4*z, 11*z*z-4*z+1, 10*z*z-6*z, -10*z*z+5*z-1, 10*z*z-6*z+1]
coeffs = [(_.subs(z,1/z) * z*z).factor() for _ in coeffs]
f2 = ('+s({0}a6-{1}(a5b+a5c)+{2}(a4b2+a4c2)+{3}a4bc+{4}a3b3+{5}(a3b2c+a3bc2)+{6}a2b2c2)'.format(*map(lambda x: x.subs(z,x_), coeffs)))
f1 = f'{z_.subs(x,x_).subs(t,t_)}s(a3-abc-{(x_+sp.S(2))/2}(a2b+ab2-2abc))2-{y_}p(a-b)2'
print(f1 + f2)

4/5s(a3-abc-2(a2b+ab2-2abc))2-68/25p(a-b)2+s(1a6-3(a5b+a5c)+3(a4b2+a4c2)+7a4bc+-2a3b3+-4(a3b2c+a3bc2)+2a2b2c2)


In [66]:
# eq = pl('s(a2(b-c)2((za-b-c)2-ybc)2)').subs(c,1).subs(b,sp.S(1)/2).subs(a,sp.S(1)/4).as_expr().subs(y,(x*(x - 2*z + 2) + z**2 - 2*z + 1)/x)
# # print(sp.solve(eq.args[-1].args[0],y)[0])
# list(sp.polys.roots(eq.subs(x,sp.S(1)/2).factor()).keys())[0]
eq = pl('s(a3+y(a2b+ab2)+zabc)').subs(c,1)
eq = pl('s(a4-a2bc+y(a3b+ab3-2a2bc)+z(a2b2-a2bc))').subs(c,1)
# eq = pl('(a2+w(b2+c2)-y(ab+ac)-zbc)').subs(c,1)
# eq2 = pl('(b2+w(a2+c2)-y(bc+ba)-zac)').subs(c,1)
# sp.solve((eq.subs(b,a).subs(a,sp.S(1)/2), eq.subs(a,sp.S(1)/4).subs(b,sp.S(1)/2), eq2.subs(a,sp.S(1)/4).subs(b,sp.S(1)/2)), (w,y,z))
# sp.solve((eq.subs(b,a).subs(a,sp.S(1)/2), eq.subs(a,sp.S(1)/4).subs(b,sp.S(1)/2)), (y,z))
a_, b_ = .25, .5
print([sp.nsimplify(_.n(30)) for _ in sp.solve((eq.subs(b,a).subs(a,sp.S(1)/2), eq.subs(a,a_).subs(b, b_)), (y,z)).values()])
print(sp.nsimplify((a_**2+b_**2+1)/(a_*b_+a_+b_)))

[-3/2, -1]
3/2


In [45]:
# poly = pl('s(45a6-155a5b-155a5c+7a4b2+831a4bc+7a4c2+286a3b3-876a3b2c-876a3bc2+886a2b2c2)s(a2+yab)-zs(a2-14/9ab)2s((a-b)2(a+b-3c)2)-ws(a3-29/17(ab2+a2b)+118/51abc)2s(a2-ab)-(45-w-2*z)s(a4-a2bc-5/3(a3b+ab3-2a2bc))2').as_expr().as_poly(a,b,c)
poly = pl('s(12a6-44a5b-44a5c-a4b2+262a4bc-a4c2+98a3b3-298a3b2c-298a3bc2+314a2b2c2)s(a2+yab)-zs(a2-3/2ab)2s((a-b)2(a+b-3c)2)-ws(a3-11/6(ab2+a2b)+47/18abc)2s(a2-ab)-(12-w-2*z)s(a4-a2bc-3/2(a3b+ab3-2a2bc)-(a2b2-a2bc))2')
result = sp.solve([monom_of(poly, _) for _ in [(7,1,0), (6,2,0),(5,3,0)]], (w,y,z))
print(result)
poly = poly.as_expr().subs(w,result[w]).subs(y,result[y]).subs(z,result[z])
# poly # == 0

{w: 216/19, y: -18/19, z: 4/57}


In [396]:
f2 = pl(('+s({0}a6-{1}(a5b+a5c)+{2}(a4b2+a4c2)+{3}a4bc+{4}a3b3+{5}(a3b2c+a3bc2)+{6}a2b2c2)'.format(*map(lambda x: ('(%s)'%x).replace('*','').replace(' ',''), coeffs)))).subs(z,x)
f1 = pl('zs(a3-abc-(x+2)/2(a2b+ab2-2abc))2-yp(a-b)2').subs(z,z_).subs(y,yroots[1])
poly = (f1 + f2).as_poly(a,b,c)
poly = (poly / (monom_of(poly, (6,0,0)))).as_poly(a,b,c)

In [510]:
# swa(poly.subs(t,sp.S(1)/12).subs(x,sp.S(7)/3))
# target: dynamic upgrading
# f(a,b,c) * s(a2-ab + xab)  >= g(a,b,c)

### Sextic Rotated Tree

$x\geqslant -1$
$$\sum a^2b^4+(x^3-3x)\sum a^3b^2c-3x(x-1)\sum a^3bc^2-3(x^3-3x^2+1)a^2b^2c^2\geqslant 0$$

In [416]:
poly = pl('s(a2b4+(x3-3x)a3b2c-3x(x-1)a3bc2-(x3-3x2+1)a2b2c2)s(ab2+(x2+x)/3abc)').as_poly(a,b,c)
# t_ = ((a*b-a)*(a-b)-(b*(a-1))**2)
# u_ = ((b*b-a*a)*(a-b) - (b-a*b)*(1-b*b))/t_
# v_ = ((a*b-a)*(1-b*b)-(b*b-a*a)*(b-b*a))/t_
# v_.subs(a,1/(x-1)**2).subs(b,1/(x-1)).factor()
u_, v_ = x, x / (x-1)
# umb = pl('((-u*v + 1)c(a2-b2+u(ab-ac)+v(bc-ab))+u((u*v - 1)(b2a-c2a)-(u2 + v)(b2c-abc)+(u + v2)(bc2-abc)))')
# umb = umb.subs(u,u_).subs(v,v_).as_expr().factor().args[-1]
# print(umb.as_poly(a,b,c))
# umb = (a**2*c - x*a*b**2  + (x*x-1)*b**2*c - x*b*c**2 - (x**2 - 2*x)*a*b*c).as_poly(a,b,c)
poly -= pl('s(a2c(a2c-xab2+(x2-1)b2c-xbc2-(x2-2x)abc)2)').as_poly(a,b,c)
coeffs = [(u*v - 1)**2, u**2 - u*v + u + v**2 + v + 1, u**4 + u**3 - 3*u**2*v - 2*u*v**3 + 3*u*v**2 + 2*u - 2*v**3 - 3*v - 1, -2*u**3*v - 2*u**3 + 3*u**2*v - 3*u*v**2 - 3*u + v**4 + v**3 + 2*v - 1, -(u**4 - 2*u**3*v - u**3 + u**2*v**2 + u**2 - 2*u*v**3 - 3*u*v + v**4 - v**3 + v**2)]
coeffs = [_.subs(u,u_).subs(v,v_).factor()*(x-1)**4/(x*x-x+1)**2*(x+1) for _ in coeffs]
poly -= pl('abcs(ua3b3+va4bc+wa3b2c+za2b3c+ra2b2c2)').subs((zip((u,v,w,z,r), coeffs))).as_poly(a,b,c)
print(poly)

Poly(0, a, b, c, domain='ZZ[x]')


In [616]:
# mild method
poly = pl('s(a2b4+ua3b2c+va3bc2+a2b2c2)s(a)-s(a(a2c-ab2+x(bc2-abc)+y(b2c-abc))2)')
p1 = monom_of(poly,(4,2,1)) 
q1 = monom_of(poly,(2,4,1))
det = (p1*q1*4 - monom_of(poly,(3,3,1))**2).factor()

#sp.solve(det.subs(x,sp.solve(det.diff(x),x)[0].factor()).factor().args[-1].as_poly(y).diff(y),y)[0]
x_ = sp.solve(det.diff(x),x)[0]

In [608]:
# dett = det.subs(x,sp.solve(det.diff(x),x)[0].factor()).factor().args[-1].subs(y,sp.sqrt(u+2)).factor()
#.subs(u,12).subs(v,-12)
# sdesmos(dett.subs(u,x).subs(v,y))
# dett = det.subs(x,sp.solve(det.diff(x),x)[0].factor()).factor().args[-1]
# q11 = q1.subs(x,x_).factor() *(u+4*y+6)**2 * (-4)  # < 0
# (dett*(y+2)**2 + q11).factor() # < 0

### Sextic Uncentered Tree

`s(a6+4a3b3-11a4bc+43/6a2b2c2)-7/8s(a3-5/3abc)2`

`s(a6+6a3b3-12a4bc+6a2b2c2)-3/4s(a3-5/3abc)2`

`s(3a6-78a4bc+66a3b3+107/3a2b2c2)`

In [10]:
poly = pqr_cyc(pl('s(a6+ua3b3+va4bc+za2b2c2)').as_poly(a,b,c))[0].subs(p,1).subs(q,(1-t)/3).as_poly(r)
det = sp.polys.discriminant(poly).as_poly(t)
dett = sp.polys.discriminant(det).factor().args[-1]

## Sextic Hexagon

Vasile:

`s(a4b2+9a4bc-48a3b2c+29a3bc2+9a2b2c2)`
`s(4a4bc+a4c2-14a3b2c+2a3bc2+7a2b2c2)`

`s(a4b2+9a4bc-48a3b2c+29a3bc2+9a2b2c2)+9s(4a4bc+a4c2-14a3b2c+2a3bc2+7a2b2c2)-6s(a3b3+7a4bc-29a3b2c+12a3bc2+9a2b2c2)`

`(s(a4b2+9a4bc-48a3b2c+29a3bc2+9a2b2c2)s(a)-9/2abcs((a2-b2+(ab-ac)+2(bc-ab))2))s(a2b)-0s(bc2((a2c-b2c)-3(a2b-abc)+5(ab2-abc))2)-0/9abc(s(2a2b-3ab2)+3abc)2+0abcs(a3b3+7a4bc-29a3b2c+12a3bc2+9a2b2c2)-39/2p(a2)s((a2-b2+(ab-ac)+2(bc-ab))2)-s(ab(a(a2b-3abc-2ac2-5b2c+9bc2)+c((a2c-b2c)-3(a2b-abc)+5(ab2-abc))+0ac(b2-bc-ba-c2+2ca))2)`

`s(b(((a2c-b2c)-3(a2b-abc)+5(ab2-abc))-5a(b2-c2+(bc-ba)+2(ca-bc)))2)`


`s(a5b2-9a4b2c+25a4bc2-20a3b3c+3a3b2c2)s(a2b+3abc)-s(ab(a(a2b-3abc-2ac2-5b2c+9bc2)+3c((a2c-b2c)-3(a2b-abc)+5(ab2-abc)))2)`

`s(a5b2-9a4b2c+25a4bc2-20a3b3c+3a3b2c2)s(ac2+3abc)-s(ab(a(a2b-3abc-2ac2-5b2c+9bc2)-c((a2c-b2c)-3(a2b-abc)+5(ab2-abc)))2)`

`s(b2c(c2-a2+(ca-cb)+2(ab-ca))2)`

`s(a5b2-2a4b3-4a4b2c+5a4bc2+a4c3+a3b3c-2a3b2c2)`

非常压缩 `s(a4b3-2a4b2c+13a4bc2-17a3b3c+5a3b2c2)` 与 `s(a4b2c+6a4bc2+a4c3-13a3b3c+5a3b2c2)`

提示 `s(a4b3-2a4b2c+13a4bc2-17a3b3c+5a3b2c2)s(a2b+31/3abc)-s(a2b2((a2b-3abc-2ac2-5b2c+9bc2)-c(a2-b2+ab-ac+2(bc-ab))+c(b2-c2+bc-ba+2(ca-bc)))2)-20p(a)s(c((a2c-b2c)-3(a2b-abc)+5(ab2-abc))2)-p(a2)s((a2-b2+ab-ac+2(bc-ab))2)` 与 `s(a4b2c+6a4bc2+a4c3-13a3b3c+5a3b2c2)s(a2)-s(a2-ab)s(a2c(-ab+ac+2b2-3bc)2)-s((abc+ac2)(a2b-a2c-ab2-b2c+2bc2)2)`

...

对应 `s(a5b2-9a4b2c+25a4bc2-20a3b3c+3a3b2c2)` 与 `s(a5c2+a4b2c+a4bc2-7a3b3c+4a3b2c2)`

提示: `s(a5b2-9a4b2c+25a4bc2-20a3b3c+3a3b2c2)s(a2b+37/7abc)-s(ab(a(a2b-3abc-2ac2-5b2c+9bc2)-15/7c((a2c-b2c)-3(a2b-abc)+5(ab2-abc)))2)-120/7p(a2)s((a2-b2+(ab-ac)+2(bc-ab))2)-384/49p(a)s(c((a2c-b2c)-3(a2b-abc)+5(ab2-abc))2)` 与 `s(a5c2+a4b2c+a4bc2-7a3b3c+4a3b2c2)s(ab2+1/3abc)-s(a3c(a2c-ab2+abc+2b2c-3bc2)2)-2p(a2)s((a2-b2+(ab-ac)+2(bc-ab))2)`

另解 `s(a5b2-9a4b2c+25a4bc2-20a3b3c+3a3b2c2)s(a2+2ab)-s(a(a3b-a2b2+2a2bc-6ab2c+2abc2-2ac3-b3c+5bc3)2)-4abcs(a)s(2a4b-6a3b2-3a3bc+5a3c2+2a2b2c)-2abcs((2ab2-ca2-a2b-bc2+c2a)2)` 

`s(a4b3-2a4b2c+13a4bc2-17a3b3c+5a3b2c2)s(a2b2+2a2bc)-s(ac4(((a2c-b2c)-3(a2b-abc)+5(ab2-abc)))2)`


In [4]:
# special case: Vasile
poly = pl('s(a4b2+9a4bc-48a3b2c+29a3bc2+9a2b2c2)+x2s(4a4bc+a4c2-14a3b2c+2a3bc2+7a2b2c2)-2xs(a3b3+7a4bc-29a3b2c+12a3bc2+9a2b2c2)')
poly = poly * pl('s(ab2-abc)')
poly = poly.as_poly(a,b,c)
poly2 = pl('s(a2c(((x-1)a-(2x-3)b+(x-2)c)(a2-b2+(ab-ac)+2(bc-ab))+((x-1)a-(x-1)b)(c2-a2+(ca-cb)+2(ab-ca)))2)').as_poly(a,b,c)
poly -= poly2
print(poly.as_expr().factor())

a*b*c*(x - 3)*(x - 1)*(2*a**2*b - 3*a**2*c - 3*a*b**2 + 3*a*b*c + 2*a*c**2 + 2*b**2*c - 3*b*c**2)**2


In [2]:
comb1 = (pl('s(((1-uv)c(a2-b2+u(ab-ac)+v(bc-ab))+u((uv-1)(b2a-c2a)-(u2+v)(b2c-abc)+(v2+u)(bc2-abc)))2)') / (u*u-u+1)).as_poly(a,b,c)
comb2 = (pl('s(((uv-1)(a2c-b2c)-(u2+v)(a2b-abc)+(v2+u)(ab2-abc))2)')).as_poly(a,b,c)
coeffs = [(u*v - 1)**2, u**2 - u*v + u + v**2 + v + 1, u**4 + u**3 - 3*u**2*v - 2*u*v**3 + 3*u*v**2 + 2*u - 2*v**3 - 3*v - 1, -2*u**3*v - 2*u**3 + 3*u**2*v - 3*u*v**2 - 3*u + v**4 + v**3 + 2*v - 1, -(u**4 - 2*u**3*v - u**3 + u**2*v**2 + u**2 - 2*u*v**3 - 3*u*v + v**4 - v**3 + v**2)]
hexagram = (pl('s(xa3b3+ya4bc+za3b2c+wa2b3c+ra2b2c2)').as_expr().subs(zip((x,y,z,w,r), coeffs))).as_poly(a,b,c)
right = (monom_of(comb1,(4,2,0)).factor() * comb2 - monom_of(comb2,(4,2,0)).factor() * comb1).factor_list()[1][0][0]
right = monom_of(hexagram, (3,3,0)) * right - monom_of(right,(3,3,0))*hexagram
right = -right.factor_list()[1][0][0]
left = (monom_of(comb1,(2,4,0)).factor() * comb2 - monom_of(comb2,(2,4,0)).factor() * comb1).factor_list()[1][0][0]
left = monom_of(hexagram, (3,3,0)) * left - monom_of(left,(3,3,0))*hexagram
left = left.factor_list()[1][0][0]
original = (left + x**2  * right - 2*x*hexagram).as_poly(a,b,c)

In [3]:
u_, v_, x_ = 3, 1, sp.S(1)
print('Left  =', poly_get_standard_form(left.subs(u,u_).subs(v,v_).as_poly(a,b,c), is_cyc = True))
print('Right =', poly_get_standard_form(right.subs(u,u_).subs(v,v_).as_poly(a,b,c), is_cyc = True))
print('Hex   =', poly_get_standard_form(hexagram.subs(u,u_).subs(v,v_).as_poly(a,b,c), is_cyc = True))
print('Mix   =', poly_get_standard_form((original).subs(x,x_).subs(u,u_).subs(v,v_).as_poly(a,b,c), is_cyc = True))

Left  = s(4a4b2+4a4bc+12a3b2c-28a3bc2+8a2b2c2)
Right = s(16a4bc+4a4c2+248a3b2c-216a3bc2-52a2b2c2)
Hex   = s(12a4bc+4a3b3+84a3b2c-96a3bc2-4a2b2c2)
Mix   = s(4a4b2-4a4bc+4a4c2-8a3b3+92a3b2c-52a3bc2-36a2b2c2)


In [4]:
border = (left + x*x * right - 2*x*hexagram).subs(c,1).as_expr()#.subs(v,2).subs(u,sp.S(2)/3)
borderdb = border.diff(b).subs(b,0).subs(a,1/x).factor()
borderda = border.diff(a).subs(b,0).subs(a,1/x).factor()
# sp.polys.nroots(borderdb)

In [71]:
# (monom_of((left + x*x * right - 2*x*hexagram),(4,1,1)) - monom_of((left + x*x * right - 2*x*hexagram),(3,3,0))).factor()

In [3]:
poly = (left + x*x * right - 2*x*hexagram) * pl('s(ab2+yabc)').as_poly(a,b,c)
poly2 = pl('s(a2c((za-rb+(vz+x(1-uv))c)(a2-b2+u(ab-ac)+v(bc-ab))+(za-wb)(c2-a2+u(ca-cb)+v(ab-ca)))2)').as_poly(a,b,c)
# z_ = sp.solve((monom_of(poly2,(1,0,2))-u*v+1),z)[0]
z_ = u*x-1
r_ = u**2*x - u*v - u + w + 1
w_ = -u**2*x + u*v + u*x + u - v + x - 2
# poly2 = poly2.subs(z, z_).as_poly(a,b,c)
# r_ = sp.solve((monom_of(poly2,(2,1,0))-u*v+1), r)[0]
# poly2 = poly2.subs(r, r_).as_poly(a,b,c)
# w_ = sp.solve((monom_of(poly2,(0,3,0)) - u*x+v-x+1), w)[0]
# poly2 = poly2.subs(w, w_).as_poly(a,b,c)
poly2 = poly2.subs(z,z_).subs(r,r_).subs(w,w_).as_poly(a,b,c)
res = poly - poly2
res2 = res - (monom_of(res,(2,6,1)).factor() * pl('abcs(ac(a2-b2+u(ab-ac)+v(bc-ab))2)')).as_poly(a,b,c)
# y_ = sp.solve((monom_of(res2,(5,3,1)) / monom_of(res2,(3,5,1)) - (u+1)**2/(v+1)**2).factor().args[-1].as_poly(y),y)[0].factor()
# # resagram = res2.subs(y,y_).as_poly(a,b,c) - ((monom_of(res2, (5,3,1)).subs(y,y_).factor() / (u**2+1)) * pl('abcs(c2(a2-b2+u(ab-ac)+v(bc-ab))2)').as_expr()).as_poly(a,b,c).factor_list()
# cuthead = (monom_of(res2, (5,3,1)).subs(y,y_).factor() / (u**2+1))
# reshead = (u*x - v + x - 1)*(u**5*x**3 - u**4*v*x**3 - u**4*v*x**2 + u**4*x**3 - 3*u**4*x**2 + u**3*v**2*x**2 - u**3*v**2*x + 2*u**3*v*x**3 - u**3*v*x**2 + 2*u**3*v*x + 2*u**3*x**3 - 2*u**3*x**2 + 3*u**3*x + u**2*v**3*x + u**2*v**3 - u**2*v**2*x**2 + u**2*v**2*x + u**2*v**2 - 7*u**2*v*x**2 + 7*u**2*v*x - u**2*v - 4*u**2*x**3 - 2*u**2*x**2 + u**2*x - u**2 - u*v**4 - u*v**3 + 3*u*v**2*x**2 + 3*u*v**2*x - 5*u*v**2 - 2*u*v*x**3 - 5*u*v*x**2 + 2*u*v*x - 5*u*v + 3*u*x**3 + 12*u*x**2 - 3*u*x - v**4 + 3*v**3*x + 6*v**3 - 3*v**2*x**2 - 15*v**2*x - 2*v**2 + v*x**3 + 12*v*x**2 + 5*v*x + 6*v - 3*x**3 - 3*x**2 - 9*x + 3)/((u*v - 1)*(u**2*x**2 - v**2 + x**2 - 1))
# (monom_of(poly,(2,6,1)) -monom_of(poly,(5,0,4))).factor() # x*(1-uv)    uv - 1    uv - 1, ux-v+x-1
# example: 1/243s(27a4b2+522a4bc+27a4c2-54a3b3-2506a3b2c+2166a3bc2-182a2b2c2)s(ab2-19/108abc)-s(a2c(a^2*b/3-a^2*c/3+a*b^2/9-28*a*b*c/27+a*c^2/3-4*b^3/3+85*b^2*c/27-11*b*c^2/9)2)-16/27abcs(ac(a2-b2+2/3(ab-ac)+2(bc-ab))2)-65/36p(a)s(c2(a2-b2+2/3(ab-ac)+2(bc-ab))2)
# y_ = sp.solve((monom_of(res2,(5,3,1)) / monom_of(res2,(3,5,1)) - (u+1)**2/(v+1)**2).factor().args[-1].as_poly(y),y)[0].factor()
y_ = (u - 1)*(u*x - v + x - 1)*(u**2 + u*v + 3*u + v**2 + 3*v + 3)/(3*(u*v - 1)*(u*x + v + x + 1)) - 1
# ker2 = (pl('abcs((u3+1)(abc-a2b)+(u2v+u2-uv-u+v+1)(a2c-abc))2')).as_expr()
# cuthead = monom_of(res2, (5,3,1)).subs(y,y_).factor() / ((u + 1)**2*(u**2 - u + 1)**2)
cuthead = -(-u**3*v*x + 2*u**2*v**2*x**2 - u**2*v**2*x + u**2*v**2 + u**2*v*x**3 - u**2*v*x + u**2*v - u**2*x**3 + u**2*x - u*v**3*x + u*v**3 + u*v**2*x**2 + 2*u*v**2*x + u*v**2 - 8*u*v*x**2 + 2*u*v*x - u*v - u*x**3 + u*x**2 + u*x - u - v**4 + v**3*x + v**2*x**2 + v**2*x - v**2 - v*x**3 - v*x**2 - 2*v*x - 2*v + 6*x**2)/((u**2 - u + 1)**2*(u*x + v + x + 1))
# resagram = (res2.subs(y,y_).as_poly(a,b,c) - (cuthead * ker2).as_poly(a,b,c))#.as_expr().factor()
reshead = (u - 1)*(u*x - v + x - 1)**2*(u**2*x - u*v*x + u*v + 2*u*x - u - v**2 + v*x - 2*v + 3*x - 3)/((u*v - 1)*(u*x + v + x + 1))
# print(resagram - (reshead * hexagram.as_expr() * a*b*c).as_poly(a,b,c))  # zero
# resagram = reshead * hexagram

# require: (u-1)(v-1) <= 0, (u - 1)*(u*x - v + x - 1) >= 0, cuthead.args[-1] >= 0, (u - 1)*(u**2*x - u*v*x + u*v + 2*u*x - u - v**2 + v*x - 2*v + 3*x - 3) >= 0

In [5]:
poly = pl('s(7a4b2+4a2b4-8a3b3-abc(8a3+24a2b-30ab2+abc))')
poly -= pl('xs((a2c-b2c-u(a2b-abc)+v(ab2-abc))2)').as_poly(a,b,c).as_expr().subs(u,1+1/sp.sqrt(2)).subs(v,1+sp.sqrt(2)).as_poly(a,b,c)
poly -= pl('ys((a2c-b2c-u(a2b-abc)+v(ab2-abc)))2').as_poly(a,b,c).as_expr().subs(u,1+1/sp.sqrt(2)).subs(v,1+sp.sqrt(2)).as_poly(a,b,c)

In [12]:

poly = (left + x*x * right - 2*x*hexagram) * pl('s(ab2+yabc)').as_poly(a,b,c)
poly2 = pl('s(a2c((za-rb+(vz+x(1-uv))c)(a2-b2+u(ab-ac)+v(bc-ab))+(za-wb)(c2-a2+u(ca-cb)+v(ab-ca)))2)').as_poly(a,b,c)
# z_ = sp.solve((monom_of(poly2,(1,0,2))-u*v+1),z)[0]
z_ = u*x-1
r_ = u**2*x - u*v - u + w + 1
w_ = -u**2*x + u*v + u*x + u - v + x - 2
# poly2 = poly2.subs(z, z_).as_poly(a,b,c)
# r_ = sp.solve((monom_of(poly2,(2,1,0))-u*v+1), r)[0]
# poly2 = poly2.subs(r, r_).as_poly(a,b,c)
# w_ = sp.solve((monom_of(poly2,(0,3,0)) - u*x+v-x+1), w)[0]
# poly2 = poly2.subs(w, w_).as_poly(a,b,c)
poly2 = poly2.subs(z,z_).subs(r,r_).as_poly(a,b,c)
res = poly - poly2
res2 = res - (monom_of(res,(2,6,1)).factor() * pl('abcs(ac(a2-b2+u(ab-ac)+v(bc-ab))2)')).as_poly(a,b,c)

In [33]:
# eq = res2.as_expr().factor().args[-1].subs(c,0).subs(b,1).subs(a,x).factor()
# eqw = sp.solve(eq.args[-1],w)[0].factor()
eq = (monom_of(res2,(5,3,1))*x**2 +  monom_of(res2,(4,4,1))*x + monom_of(res2,(3,5,1))).factor()
eqw = list(sp.polys.roots(eq.args[-1].as_poly(w)))[0].factor()
ker_ = monom_of(res,(2,6,1)).subs(w,eqw).factor()
ker = ker_.args[-1]

In [17]:
swa((((pl('((za-rb+(vz+x(1-uv))c)(a2-b2+u(ab-ac)+v(bc-ab))+(za-wb)(c2-a2+u(ca-cb)+v(ab-ca)))').as_expr().subs(z,z_).subs(r,r_).subs(w,w_).subs(u,1).subs(v,2).subs(x,sp.S(1)/2))).as_poly(a,b,c).as_expr()))

a^2*b-a^2*c/2-a*b^2/2-3*a*b*c/2+a*c^2-2*b^3+5*b^2*c-5*b*c^2/2


'a^2*b-a^2*c/2-a*b^2/2-3*a*b*c/2+a*c^2-2*b^3+5*b^2*c-5*b*c^2/2'

In [237]:
# (res - (monom_of(res,(2,6,1)).factor() * pl('abcs(ac(a2-b2+u(ab-ac)+v(bc-ab))2)')).as_poly(a,b,c)).as_expr().subs(u,1).factor()
# res.subs(x,0).factor()

In [72]:
# sp.polys.roots(monom_of(pl('1/9s(a4b2+42a4bc+9a4c2-6a3b3-130a3b2c+46a3bc2+38a2b2c2)s(ab2+abc)-s(a2c(a^2*b/3-a^2*c-5*a*b^2/3+32*a*b*c/9+a*c^2/3+2*b^3-35*b^2*c/9+b*c^2/3+wb(b2-c2+2/3(bc-ba)+2(ca-bc)))2)'),(2,6,1)))
# sp.polys.roots(.as_poly(y))
# det = sp.polys.discriminant(pl('1/9s(a4b2+42a4bc+9a4c2-6a3b3-130a3b2c+46a3bc2+38a2b2c2)s(ab2+yabc)-s(a2c(a^2*b/3-a^2*c-5*a*b^2/3+32*a*b*c/9+a*c^2/3+2*b^3-35*b^2*c/9+b*c^2/3+wb(b2-c2+2/3(bc-ba)+2(ca-bc)))2)').as_expr().factor().args[-1].subs(c,0).subs(b,1).subs(w,4/sp.sqrt(3)-2).as_poly(a).as_expr().factor(extension = sp.sqrt(3)).args[-1].as_poly(a)).factor().args[-1].as_poly(y)
# sp.simplify(list(sp.polys.roots(det))[0])

In [ ]:
pl('1/9s(a4b2+42a4bc+9a4c2-6a3b3-130a3b2c+46a3bc2+38a2b2c2)s(ab2+abc)-s(a2c(a^2*b/3-a^2*c-5*a*b^2/3+32*a*b*c/9+a*c^2/3+2*b^3-35*b^2*c/9+b*c^2/3+wb(b2-c2+2/3(bc-ba)+2(ca-bc)))2)')

In [274]:
ppp = pl('s(c2(a2-b2+u(ab-ac)+v(bc-ab))2)').as_poly(a,b,c)
monom_of(ppp,(4,2,0))# / monom_of(ppp,(2,4,0))

u**2 + 1

In [75]:
# sp.solve((monom_of(poly,(4,2,0)),monom_of(poly,(2,4,0))))
k = sp.sqrt(2)+1
# sp.simplify(k**3-3*k)
poly = pl('s(7a4b2+4a2b4-8a3b3-abc(8a3+24a2b-30ab2+abc))')
# poly += 4 * pl('')
# poly.as_expr().factor(extension = sp.sqrt(2))
# poly -= pl('xs(c2(a2-b2+u(ab-ac)+v(bc-ab))2)').as_poly(a,b,c).as_expr().subs(u,1+1/sp.sqrt(2)).subs(v,1+sp.sqrt(2)).as_poly(a,b,c)
# # monom_of(poly,(3,3,0))
# sp.simplify(poly.as_expr().subs(x, 8/(4+3*sp.sqrt(2))).as_poly(a,b,c).as_expr()).as_poly(a,b,c)
eq0 = pl('((ab2-abc-x(a2b+cb2-2abc)+y(a2c+bc2-2abc)+z(ac2-abc)))')
eq1 = eq0.subs(b,1).subs(c,2).subs(a,sp.sqrt(2))
eq2 = eq0.subs(a,1).subs(b,2).subs(c,sp.sqrt(2))
eq3 = eq0.subs(c,1).subs(a,2).subs(b,sp.sqrt(2))
eq0 = pl('s((ab2-abc-x(a2b+cb2-2abc)+y(a2c+bc2-2abc)+z(ac2-abc))2)')
poly -= x*eq0.as_poly(a,b,c).as_expr().subs(sp.solve((eq1,eq2,eq3))).as_poly(a,b,c)
# poly -= (20*sp.sqrt(2)-1)/17 * pl('s(a4b2+(k3-3k)a2b3c-3k(k-1)a3b2c-(k3-3k2+1)a2b2c2)').subs('k',k).as_poly(a,b,c)
# poly -= (8-2*sp.sqrt(2))/5 * pl('s(a4c2+(k3-3k)a2bc3-3k(k-1)a3bc2-(k3-3k2+1)a2b2c2)').subs('k',1+1/sp.sqrt(2)).as_poly(a,b,c)

In [94]:
# x_ = (sp.solve(monom_of(poly,(3,3,0)),x)[0])
# sdesmos(monom_of(poly,(4,2,0)).as_poly(x).as_expr().subs(x,x_).subs(z,x).factor(extension = sp.sqrt(2)))
# sdesmos(monom_of(poly,(2,4,0)).as_poly(x).as_expr().subs(x,x_).subs(z,x).factor(extension = sp.sqrt(2)))

In [92]:
# swa(pl('s(a4c2+(k3-3k)a2bc3-3k(k-1)a3bc2-(k3-3k2+1)a2b2c2)').subs('k',1+1/sp.sqrt(2)).as_expr().n(20))
# swa(poly.as_expr().n(10))
# sdesmos(pl('s(7a4b2+4a2b4-8a3b3-abc(8a3+24a2b-30ab2+abc))-3/9.74264068713834s(9.74264068713834a4b2-99.7903679014609a3b2c+66.5269119340601a3bc2+23.5208152802940a2b2c2)').subs(c,1).subs(a,x).subs(b,y))

In [93]:
# poly_get_standard_form(pl('s(7a4b2+4a2b4-8a3b3-abc(8a3+24a2b-30ab2+abc))').as_expr().subs(c,-c).as_poly(a,b,c))
# sdesmos(pl('s(7a4b2+4a2b4-8a3b3-abc(8a3+24a2b-30ab2+abc))').as_expr().subs(c,-1).subs(a,x).subs(b,y))

#### Method2: multiply s(a)


In [192]:
y_ = (-u*x + 1)/(v - x)
poly2 = (pl('s(a((a+zc)(a2-b2+u(ab-ac)+v(bc-ab))+(a+yb)(c2-a2+u(ca-cb)+v(ab-ca)))2)').as_expr()).subs(y,y_).as_poly(a,b,c)
res = original * pl('s(a)') - (x-v)**2 * poly2
# z__ = [(-u*v + v - x + 1)/(u*(v - x)), (u*v + v - x - 1)/(u*(v - x))]
# z_ = z__[1]
# # z_ = (u**2*x + u*v*x + u*v - u + v**2 - v*x - x - 1)
# res = res.subs(z,z_).as_poly(a,b,c)#factor()

In [191]:
sp.solve(monom_of(res,(5,2,0)).factor(),y)
# monom_of(res,(4,3,0)).factor()

[(-u*x + 1)/(v - x), (2*u*v - u*x - 1)/(v - x)]

#### Method3: multiply s(ab)


In [131]:
y_ = (-u*x + 1)/(v - x)
# eq = pl('((a+zc)(a2-b2+u(ab-ac)+v(bc-ab))+(a+yb)(c2-a2+u(ca-cb)+v(ab-ca)))')
# y_ = sp.solve(monom_of(eq,(1,2,0)) / monom_of(eq,(2,1,0)) + x, y)[0]
poly2 = (pl('s(ab((a+zc)(a2-b2+u(ab-ac)+v(bc-ab))+(a+yb)(c2-a2+u(ca-cb)+v(ab-ca)))2)').as_expr()).subs(y,y_).as_poly(a,b,c)
res = original * pl('s(ab)') - (x-v)**2 * poly2
z_ = (u**2*x + u*v*x + u*v - u + v**2 - v*x - x - 1)/(2*(v - x))
# res2 = res.subs(z, z_).as_poly(a,b,c)
# res2.as_expr().factor()
res2_coeff = (u**4*x**2 + 2*u**3*v*x**2 + 2*u**3*v*x - 4*u**3*x**2 - 2*u**3*x + u**2*v**2*x**2 + 4*u**2*v**2*x + u**2*v**2 + 6*u**2*v*x**2 + 2*u**2*v*x - 2*u**2*v - 6*u**2*x**2 + 6*u**2*x + u**2 + 2*u*v**3*x + 2*u*v**3 - 2*u*v**2*x**2 + 2*u*v**2*x + 6*u*v**2 - 6*u*v*x**2 - 22*u*v*x - 6*u*v - 4*u*x**2 + 6*u*x - 2*u + v**4 - 2*v**3*x - 4*v**3 + v**2*x**2 + 6*v**2*x - 6*v**2 - 2*v*x**2 + 6*v*x - 4*v + x**2 + 14*x + 1)/(-4)
res2_coeff = res2_coeff.as_poly(x)
res2 = a**4*b + a**4*c + a**3*b**2*u**2 - 2*a**3*b**2*v - 2*a**3*b*c*u*v - 2*a**3*c**2*u + a**3*c**2*v**2 - 2*a**2*b**3*u + a**2*b**3*v**2 - a**2*b**2*c*u**2 + 2*a**2*b**2*c*u*v + 2*a**2*b**2*c*u - a**2*b**2*c*v**2 + 2*a**2*b**2*c*v - 2*a**2*b**2*c - a**2*b*c**2*u**2 + 2*a**2*b*c**2*u*v + 2*a**2*b*c**2*u - a**2*b*c**2*v**2 + 2*a**2*b*c**2*v - 2*a**2*b*c**2 + a**2*c**3*u**2 - 2*a**2*c**3*v + a*b**4 - 2*a*b**3*c*u*v - a*b**2*c**2*u**2 + 2*a*b**2*c**2*u*v + 2*a*b**2*c**2*u - a*b**2*c**2*v**2 + 2*a*b**2*c**2*v - 2*a*b**2*c**2 - 2*a*b*c**3*u*v + a*c**4 + b**4*c + b**3*c**2*u**2 - 2*b**3*c**2*v - 2*b**2*c**3*u + b**2*c**3*v**2 + b*c**4
res2 = res2.as_poly(a,b,c) # res2 = pl('s(c(a2-b2+u(ab-ac)+v(bc-ab))2)').as_poly(a,b,c)

# require res2_coeff >= 0

In [149]:
# res2_coeff.subs(u,1/2).subs(v,3).subs(x,4)
# sp.polys.nroots(res2_coeff.subs(u,3).subs(v,5))
sdesmos([_.factor() for _ in res2_coeff.all_coeffs()][0].subs(u,x).subs(v,y))

-\frac{x^{4}}{4}-\frac{x^{3}y}{2}+x^{3}-\frac{x^{2}y^{2}}{4}-\frac{3x^{2}y}{2}+\frac{3x^{2}}{2}+\frac{xy^{2}}{2}+\frac{3xy}{2}+x-\frac{y^{2}}{4}+\frac{y}{2}-\frac{1}{4}


'-\\frac{x^{4}}{4}-\\frac{x^{3}y}{2}+x^{3}-\\frac{x^{2}y^{2}}{4}-\\frac{3x^{2}y}{2}+\\frac{3x^{2}}{2}+\\frac{xy^{2}}{2}+\\frac{3xy}{2}+x-\\frac{y^{2}}{4}+\\frac{y}{2}-\\frac{1}{4}'

### Sextic Hexagon Full

In [128]:
poly = pl('s(ab(a2+zab-mb2-(z+1-m)bc-x(ac-bc)-y(c2-bc))2)').as_poly(a,b,c)
# sp.solve((q - t) / (p - t) + 2*sp.symbols('m'),t)
ker = (poly - pl('s(ab(a2+zab-mb2)2)').as_poly(a,b,c)).as_expr().factor().args[-1]#.as_poly(a,b,c)
border = ker.subs(c,1).subs(a,0).subs(b,a).as_poly(a)
border = pl('-59a3+171a2+54a-59') - border
det = sp.polys.discriminant((a*x**3+b*x**2+c*x+a).as_poly(x)).factor()
det = det.subs(zip((a,b,c), border.all_coeffs()[:-1]))#.factor()

In [2]:
# (poly - pl('s(ab(a2+zab-mb2)2)').as_poly(a,b,c)).as_expr().factor()
# poly.subs('m',2).subs(z,-3).subs(x,0).subs(y,0)
# swa(border.all_coeffs()[0].subs('m',2).subs(z,-3))#.subs(x,0).subs(y,1))
# sp.polys.discriminant((border.subs('m',2).subs(z,-3).subs(x,0).subs(y,0).subs(c,1).subs(b,0)))

In [133]:
[_.subs('m',2).subs(z,-3).factor() for _ in border.all_coeffs()]

[6*x - y**2 + 4*y - 43,
 -x**2 - 4*x*y - 16*x - y**2 - 14*y + 147,
 -x**2 + 2*x*y + 2*x + 2*y**2 + 16*y + 78,
 6*x - y**2 + 4*y - 43]

In [6]:
# (border.all_coeffs()[1] - border.all_coeffs()[2]).factor().as_poly(x)#.subs(x,sp.solve(border.all_coeffs()[0],x)[0]).factor().as_poly(y)

In [148]:
# sp.expand(det.subs(x,sp.solve(border.all_coeffs()[0],x)[0]+t))

In [49]:
# poly = pl('s(a4b2+3a4bc+9a4c2-6a3b3-25a3bc2+18a2b2c2)s(a)-s(a(a2b-3ab2+2abc-3a2c+ac2+2abc-x(bc2-abc)-y(b2c-abc))2)')
# m_, p_, n_, q_ = [monom_of(poly,(5-i,i+1,1)) for i in range(4)]
# det = 3*m_*(m_+n_) - (p_**2+p_*q_+q_**2)
# sdesmos(det)
# poly = pl('(a3b-3ab3-2a2b2+4a2bc-xabc(c-a)+yabc(b-a))').as_expr()
# eq1 = poly.subs(zip((a,b,c),((0.643104132107791, 0.198062264195162, 1)))).n(20)
# eq2 = poly.subs(zip((b,c,a),((0.643104132107791, 0.198062264195162, 1)))).n(20)
# sp.solve((eq1,eq2),(x,y))
# sp.polys.nroots(x**4+8*x**3+8-54*x**2+69*x)
# pl('s(a4b2+3a4bc+9a4c2-6a3b3-25a3bc2+18a2b2c2)s(a2c2+xa2bc)-s(a2c2((2a+b+c)(a2-ab-ac-b2+2bc)+(2a-b)(c2-ca-cb-a2+2ab))2)').as_expr().factor().args[-1].subs(c,0).subs(b,1).as_poly(a)
# pl(' s(a4b2+9a4bc-48a3b2c+29a3bc2+9a2b2c2)s(a)-9/2abcs((a2-b2+(ab-ac)+2(bc-ab))2)-s(a(a2b-ac2-0abc+x(bc2-abc)+y(b2c-abc))2)').as_poly(a,b,c)

## Septic

In [79]:
# 伴随 (uv-1)(a2c-b2c)-(u2+v)(a2b-abc)+(v2+u)(ab2-abc)
# 雨伞 ((1-uv)c(a2-b2+u(ab-ac)+v(bc-ab))+u((uv-1)(b2a-c2a)-(u2+v)(b2c-abc)+(v2+u)(bc2-abc)))
# 镰刀 ((1-uv)c(a2-b2+u(ab-ac)+v(bc-ab))+u((uv-1)(b2a-c2a)-(u2+v)(b2c-abc)+(v2+u)(bc2-abc))+(uv-1)b(b2-c2+u(bc-ba)+v(ca-bc)))
# 菜刀 ((1-uv)c(a2-b2+u(ab-ac)+v(bc-ab))+u((uv-1)(b2a-c2a)-(u2+v)(b2c-abc)+(v2+u)(bc2-abc))-(u2+v)b(b2-c2+u(bc-ba)+v(ca-bc)))
# 战锤 
u_, v_ = 1, sp.S(4)/2
print('伴随', f'(c(a2-b2+{u_}(ab-ac)+{v_}(bc-ab)))')
print('雨伞', f'(({1-u_*v_})c(a2-b2+{u_}(ab-ac)+{v_}(bc-ab))+{u_}({u_*v_-1}(b2a-c2a)-{u_*u_+v_}(b2c-abc)+{v_*v_+u_}(bc2-abc)))')
print('镰刀', f'(({1-u_*v_})c(a2-b2+{u_}(ab-ac)+{v_}(bc-ab))+{u_}({u_*v_-1}(b2a-c2a)-{u_*u_+v_}(b2c-abc)+{v_*v_+u_}(bc2-abc))+{u_*v_-1}b(b2-c2+{u_}(bc-ba)+{v_}(ca-bc)))')
print('菜刀', f'(({1-u_*v_})c(a2-b2+{u_}(ab-ac)+{v_}(bc-ab))+{u_}({u_*v_-1}(b2a-c2a)-{u_*u_+v_}(b2c-abc)+{v_*v_+u_}(bc2-abc))+{u_**2+v_}b(b2-c2+{u_}(bc-ba)+{v_}(ca-bc)))')
monom_of(pl('((1-uv)c(a2-b2+u(ab-ac)+v(bc-ab))+u((uv-1)(b2a-c2a)-(u2+v)(b2c-abc)+(v2+u)(bc2-abc))+(u2+v)b(b2-c2+u(bc-ba)+v(ca-bc)))'),(0,1,2))

伴随 (c(a2-b2+1(ab-ac)+2(bc-ab)))
雨伞 ((-1)c(a2-b2+1(ab-ac)+2(bc-ab))+1(1(b2a-c2a)-3(b2c-abc)+5(bc2-abc)))
镰刀 ((-1)c(a2-b2+1(ab-ac)+2(bc-ab))+1(1(b2a-c2a)-3(b2c-abc)+5(bc2-abc))+1b(b2-c2+1(bc-ba)+2(ca-bc)))
菜刀 ((-1)c(a2-b2+1(ab-ac)+2(bc-ab))+1(1(b2a-c2a)-3(b2c-abc)+5(bc2-abc))+3b(b2-c2+1(bc-ba)+2(ca-bc)))


0

In [342]:
print(pl('((1-uv)c(a2-b2+u(ab-ac)+v(bc-ab))+u((uv-1)(b2a-c2a)-(u2+v)(b2c-abc)+(v2+u)(bc2-abc))+(u2+v)b(b2-c2+u(bc-ba)+v(ca-bc)))').as_poly(a,b,c))

Poly((-u*v + 1)*a**2*c + (-u**3 + u**2*v - u*v - u)*a*b**2 + (u**3 - u**2 + u*v + u + v**2 - v)*a*b*c + (u**2 + v)*b**3 + (-u**2*v + u*v - v**2 - 1)*b**2*c, a, b, c, domain='ZZ[u,v]')


### Septic Remainder

$x\in\mathbb R$

$$\sum a^3(ac-b^2)(ac-b^2-3bc)+(x^2+4x)\sum (a^3b^3c-a^3b^2c^2)-2x\sum (a^2b^4c - a^3b^2c^2)\geqslant 0$$

$$\begin{aligned}\sum a\cdot f(a,b,c) & =  \sum ac\left[a^2c-bc^2+abc-(x+2)ab^2+(x+1)b^2c\right]^2 \\ & + 
\sum \left[a^3c - ab^3+b^2c^2-a^2b^2-abc^2-(x+1)a^2bc+(x+2)ab^2c\right]^2
\end{aligned}
$$
`s(a)(s(a(a2c-ab2)(a2c-ab2-3abc))+5s(a3b3c-a3b2c2)-2s(a2b4c-a3b2c2))-s(ac(a2c-3ab2+abc+2b2c-bc2)2)-s((a3c-ab3+b2c2-a2b2-2abc(a-b)-abc(c-b))2)/2`

In [255]:
poly1, poly2 = pqr_cyc(pl('s(a(a2c-ab2)(a2c-ab2-3abc))+ys(a3b3c-a3b2c2)-xs(a2b4c-a3b2c2)').as_poly(a,b,c))
poly = poly1**2 - poly2**2 * pqr_ker()
poly = poly.subs(p,1).subs(q,(1-t)/3).as_poly(r)#.subs(x,5).subs(y,2)
# det = sp.polys.discriminant(poly)
# det.factor()
poly = poly.subs(y,x*x/4+2*x).as_poly(r)
# det = sp.polys.discriminant((a*x**4+b*x**3+c*x**2+u*x+v).as_poly(x)).subs(zip((a,b,c,u,v), poly.all_coeffs()))
# det = det.factor()
det = -t**2*(t - 1)**7*(t*x + 3*t - x - 6)**2*(t*x**2 + 12*t*x + 36*t - x**2 - 12)**2*(t*x**2 + 8*t*x + 24*t - x**2 + 4*x + 12)**2*(4*t**6*x**8 + 192*t**6*x**7 + 4032*t**6*x**6 + 48384*t**6*x**5 + 362880*t**6*x**4 + 1741824*t**6*x**3 + 5225472*t**6*x**2 + 8957952*t**6*x + 6718464*t**6 - 21*t**5*x**8 - 912*t**5*x**7 - 16872*t**5*x**6 - 175968*t**5*x**5 - 1156176*t**5*x**4 - 5063040*t**5*x**3 - 14992128*t**5*x**2 - 28118016*t**5*x - 25380864*t**5 + 45*t**4*x**8 + 1752*t**4*x**7 + 28656*t**4*x**6 + 248544*t**4*x**5 + 1347408*t**4*x**4 + 4890240*t**4*x**3 + 12465792*t**4*x**2 + 23348736*t**4*x + 27869184*t**4 - 50*t**3*x**8 - 1728*t**3*x**7 - 25392*t**3*x**6 - 178848*t**3*x**5 - 745200*t**3*x**4 - 1873152*t**3*x**3 - 2512512*t**3*x**2 - 456192*t**3*x - 4997376*t**3 + 30*t**2*x**8 + 912*t**2*x**7 + 12480*t**2*x**6 + 76608*t**2*x**5 + 267984*t**2*x**4 + 452736*t**2*x**3 - 203904*t**2*x**2 - 5764608*t**2*x - 7651584*t**2 - 9*t*x**8 - 240*t*x**7 - 3240*t*x**6 - 21312*t*x**5 - 91584*t*x**4 - 200448*t*x**3 - 445824*t*x**2 + 248832*t*x + 2426112*t + x**8 + 24*x**7 + 336*x**6 + 2592*x**5 + 14688*x**4 + 51840*x**3 + 214272*x**2 + 290304*x + 1016064)/25389989167104

In [300]:
poly = pl('s(a)(s(a(a2c-ab2)(a2c-ab2-3abc))+(x2/4+2x)s(a3b3c-a3b2c2)-xs(a2b4c-a3b2c2))').as_poly(a,b,c)
poly -= pl('s(ac(a2c-(x/2+2)ab2+abc-bc2+(x/2+1)b2c)2)').as_poly(a,b,c)
poly -= pl('s((a3c-ab3+b2c2-a2b2-(x/2+1)abc(a-b)-abc(c-b))2)/2').as_poly(a,b,c)
print(poly)

Poly(0, a, b, c, domain='QQ[x]')


In [325]:
m, n = sp.symbols('m n')
det = (3*m*(m+n-(x*x+4*x+3)) - p*p - p*(q + 2*x+3) - (q+2*x+3)**2).as_poly(x)
# x_ = sp.solve(det.diff(),x)[0].factor()
# print(x_)
# print((det.as_expr().subs(x,x_).factor().args[-1] -p**2).factor()) #.subs(m,2).subs(p,-16).subs(n,39).subs(q,-1)
det = 4*p*(q + 2*x+3)- (n-(x*x+4*x+3))**2
# det.diff(x).as_poly(x).subs(x,-2)
# det.as_poly(x)
# sp.polys.discriminant(det.as_poly(x)).factor()
det

4*p*(q + 2*x + 3) - (n - x**2 - 4*x - 3)**2

(forever豪3)
$$\begin{aligned}&\quad\ \sum a\left[24abc\sum a^3+\sum a^2b\sum a^3-243abc\sum a^2b\right]\\ &
=\sum b\left[a^3-ab^2+w(b^2c-bc^2)+ \frac{w^2-4}{2}(b^2c-a^2b)-x(a^2c-abc)+y(ac^2-abc)\right]^2
\\ & + \frac{(2-w)^3(w+2)}{4}\sum c\left[(uv-1)(a^2c-b^2c)-(u^2+v)(a^2b-abc)+(v^2+u)(ab^2-abc)\right]^2
\\ & + (w+x+14)abc\sum \left[a^2-b^2+u(ab-ac)+v(bc-ab)\right]^2
\end{aligned}
$$

其中 $u = 3\sqrt 3 - 4$, $\ v = 2$, $\ w = \frac{3-\sqrt{49-12\sqrt 3}}{2}$, 
$x = -\frac{11}{2}+\frac{3\sqrt{3}}{4}+\frac{\left(4+3\sqrt{3}\right)\sqrt{49-12\sqrt{3}}}{4}$,
$y = -\frac{63}{4}+\frac{5\sqrt{49-12\sqrt{3}}}{4}+\frac{15\sqrt{3}}{2}$

In [140]:
import scipy
poly =  pl('(24abcs(a)3+s(a2b)s(a)3-243abcs(a2b))s(a)-s(b(a3+u(b2c-abc)-(u2-2u-4)/2(bc2-abc)-ab2-(u2-4)/2(a2b-abc)-x(a2c-abc)+y(ac2-abc))2)').as_poly(a,b,c)
poly -= pl('(2-w)3(w+2)/4s(c((uv-1)(a2c-b2c)-(u2+v)(a2b-abc)+(v2+u)(ab2-abc))2)*x').as_expr().subs(x,1/(u*v-1)**2).subs(v,2).subs(u,(3*sp.sqrt(3)-4)).subs(w,u).as_poly(a,b,c)
m_, p_, n_, q_ = [monom_of(poly,(5-i,i+1,1)) for i in range(4)]
det = (3*m_*(m_+n_) - (p_**2+p_*q_+q_**2)) #/ m_ / m_
det = det.factor()#.args[-1]
# # sp.nsolve((det.diff(x), det.diff(y)), (x,y), (4,4))
# def f(_):
#     return -det.subs(x,_[0]).subs(y,_[1])
# scipy.optimize.shgo(f, [(-10,10),(-10,10)])

In [297]:
# (monom_of(poly,(4,3,0))-0*monom_of(poly,(3,4,0))).factor()
# def f(_):
#     return -det.args[-1].subs(u,_[0]).subs(x,_[1]).subs(y,_[2])
# scipy.optimize.shgo(f, [(-3,2),(-10,10),(-10,10)])

In [437]:
# dett = det.args[-1].n(1000)
# sol = sp.nsolve([dett.diff(_) for _ in (u,x,y)], (u,x,y), [-1.15648242,  8.01231868,  3.88024168], prec = 1000)
# dett.subs(zip((u,x,y), sol))

In [690]:
from mpmath import mpf, mp, pslq
pslq1 = mpf(3)**(mpf(1)/2)
pslq2 = (49-12*mpf(3)**(mpf(1)/2))**(mpf(1)/2)
pslq3 = pslq1*pslq2
mp.dps = 50
pslq([1, pslq1, pslq2, pslq3, sol[2]])

r_ = sp.solve(x**4-6*x**3-11*x**2+60*x+73)[0]
x__3 = (((3*sp.sqrt(3)+4)*sp.sqrt(49-12*sp.sqrt(3))+3*sp.sqrt(3)-22)/4)
y__3 = (5*sp.sqrt(49-12*sp.sqrt(3))+30*sp.sqrt(3)-63)/4
print([(_ - __).n(100) for _, __ in zip(sol, (r_, x__3, y__3))])

[0.e-208, 0.e-208, 0.e-208]


$$\sum a^3(a-2b)^4\geqslant \sum a^4bc^2$$

猜的:
$$\prod (a-b)\sum b^{n+1}c^{n+1}(c-b)\sum (a^2-2ab) +{4\prod a^2\sum b^nc^n(a-b)(a-c) }\geqslant 0$$

等价于

$$S_n = \sum b^nc^n(a-b)(a-c)\left(a(b+c) - (b-c)^2\right)^2\geqslant 0$$

实际上

$$S_{n+1}=\sum ab \cdot S_n -  abc\sum a\cdot  S_{n-1}+a^2b^2c^2\cdot  S_{n-2}$$

旋转:

$$S_n' = \sum a^n(a-b)(a-c)\left(bc(b+c)-a(b-c)^2\right)^2$$

$$\left[\sum (a^2-2ab)\right]^2\sum ab +2abc\sum a \sum (a^2-2ab)+4a^2b^2c^2
=\sum ab\left[(a-b)^2-c(a+b)-\frac{3-3\sqrt 3}{3}c(a+b-c)\right]^2
$$

In [83]:
from mpmath import mpf, mp, pslq
# poly = pqr_cyc(pl('s(a5b+a5c-4a4b2-5a4bc-4a4c2+6a3b3+4a3b2c+4a3bc2+4/3a2b2c2)'))[0].subs(p,1).subs(q,(1-t*t)/3).as_poly(r)
# det = sp.polys.discriminant(poly).factor()
# det
# pl('(s(a5b+a5c-4a4b2-5a4bc-4a4c2+6a3b3+4a3b2c+4a3bc2+4/3a2b2c2)s(a2+9ab)-s(bc(b+c-a)2(a(b+c)-(b-c)2)2))/7').factor_list()
poly = pl('s(a5b+a5c-4a4b2-5a4bc-4a4c2+6a3b3+4a3b2c+4a3bc2+4/3a2b2c2)-s(ab(a2-2ab+b2-x(ac+bc)+yc2)2)')
eq1 = (monom_of(poly,(4,1,1)) + monom_of(poly,(3,2,1)))
eq2 = (monom_of(poly,(2,2,2)) +6*monom_of(poly,(4,1,1)))
# sol = sp.nsolve((eq1, eq2),(x,y),(1.43,0.43), n = 800)
# # sdesmos(eq1)
# mp.dps = 50
# pslq([1,sol[0],sol[0]**2,sol[0]**3,sol[0]**4])
x__ = sp.simplify(sp.solve((eq1,eq2))[0][x])
y__ = sp.simplify(sp.solve((eq1,eq2))[0][y])
print(x__.n(5), y__.n(5))
rem = poly.subs(x,x__).subs(y,y__).as_poly(a,b,c) - pl('s(2ab-a2)s(a)abc') *(5-2* sp.sqrt(3))/3
rem = rem.as_poly(a,b,c)

1.4226 0.42265


In [218]:
# import numpy as np
# np.array(part2.as_poly(a,b,c).monoms()).sum(axis = 1)
# det = ((x*x+y*y+1-2*x-2*y-2*x*y+4*x*x*y/(y-1)**2).factor())
# pl('p(a-b)2s(a2-2ab)+4p(a2)s((a-b)(a-c))-s((a-b)2(a-c)2(2a(b+c)-(b-c)2)2)/6').factor_list()
# pl('s(a2-ab)(p(a-b)s(b2c2(c-b))s(a2-2ab)+4p(a2)s(bc(a-b)(a-c)))-s(b2c2(a-b)2(a-c)2(a(b+c)-(b-c)2)2)').factor_list()
# pl('s(a6(a-b)(a-c))s(a2-ab)-s(a6(a-b)2(a-c)2)').factor_list()

In [217]:
# pl('s(a2-ab)(p(a-b)s(bc(c-b))s(a2-2ab)+4p(a2)s((a-b)(a-c)))-s(bc(a-b)2(a-c)2(a(b+c)-(b-c)2)2)').factor_list()
# pl('s(a2-ab)(p(a-b)s(b2c2(c-b))s(a2-2ab)+4p(a2)s(bc(a-b)(a-c)))-s(b2c2(a-b)2(a-c)2(a(b+c)-(b-c)2)2)').factor_list()
# pl('s(a2b2-a2bc)(p(a-b)s(b3c3(c-b))s(a2-2ab)+4p(a2)s(b2c2(a-b)(a-c)))-s(b4c4(a-b)2(a-c)2(a(b+c)-(b-c)2)2)').factor_list()
# pl('s(a2b2-a2bc)(p(a-b)s(b4c4(c-b))s(a2-2ab)+4p(a2)s(b3c3(a-b)(a-c)))-s(b5c5(a-b)2(a-c)2(a(b+c)-(b-c)2)2)').factor_list()
# pl('s(b2c2(a-b)(a-c))(p(a-b)s(b5c5(c-b))s(a2-2ab)+4p(a2)s(b4c4(a-b)(a-c)))-s(b7c7(a-b)2(a-c)2(a(b+c)-(b-c)2)2)').factor_list()

# pl('s(a2-ab)(p(a-b)s(b3c3(c-b))s(a2-2ab)+4p(a2)s(b2c2(a-b)(a-c)))-s(b3c3(a-b)2(a-c)2(a(b+c)-(b-c)2)2)').factor_list()
# pl('s(ab)s(b2c2(a-b)(a-c)(a(b+c)-(b-c)2)2)-s(b3c3(a-b)(a-c)(a(b+c)-(b-c)2)2)-abcs(a)s((a-b)2(a-c)2(a(b+c)-(b-c)2)2)/3').factor_list()

In [205]:
# poly = a**7*b**3 + a**7*b**2*c + a**7*b*c**2 + a**7*c**3 - 4*a**6*b**4 - 5*a**6*b**3*c - 5*a**6*b**2*c**2 - 5*a**6*b*c**3 - 4*a**6*c**4 + 6*a**5*b**5 + 4*a**5*b**4*c + 4*a**5*b**3*c**2 + 4*a**5*b**2*c**3 + 4*a**5*b*c**4 + 6*a**5*c**5 - 4*a**4*b**6 + 4*a**4*b**5*c + 4*a**4*b**4*c**2 + 4*a**4*b**3*c**3 + 4*a**4*b**2*c**4 + 4*a**4*b*c**5 - 4*a**4*c**6 + a**3*b**7 - 5*a**3*b**6*c + 4*a**3*b**5*c**2 + 4*a**3*b**4*c**3 + 4*a**3*b**3*c**4 + 4*a**3*b**2*c**5 - 5*a**3*b*c**6 + a**3*c**7 + a**2*b**7*c - 5*a**2*b**6*c**2 + 4*a**2*b**5*c**3 + 4*a**2*b**4*c**4 + 4*a**2*b**3*c**5 - 5*a**2*b**2*c**6 + a**2*b*c**7 + a*b**7*c**2 - 5*a*b**6*c**3 + 4*a*b**5*c**4 + 4*a*b**4*c**5 - 5*a*b**3*c**6 + a*b**2*c**7 + b**7*c**3 - 4*b**6*c**4 + 6*b**5*c**5 - 4*b**4*c**6 + b**3*c**7

# # sdesmos(poly.subs(c,1).subs(a,x).subs(b,y))
# swa(poly)

### Windmill

`s(a(c(b2-a2+ab+ac-2bc)+b(c2-b2+bc+ba-2ca))2)`

## Rotated Schur

`s(a(a+b)(a(a+b)-b(b+c))(a(a+b)-c(c+a)))`

`s((a-2b)2((a-2b)2-(b-2c)2)((a-2b)2-(c-2a)2))`

`s((a+b-3/2c)2((a+b-3/2c)2-(b+c-3/2a)2)((a+b-3/2c)2-(c+a-3/2b)2))` 多取等


rep = '(a-b+c)2+(2b-c)2' 

rep = '(a-xb+c)2+(b-xc)2'  有取等

rep = '(a2+b2-ab+bc)+x(b2+c2-bc+ca)' VASILE


取 $x=(a+b-c)(a+c-b)$, $y =(b+c-a)(b+a-c)$, $z =(c+a-b)(c+b-a)$, 则
$$\frac 14\sum_{cyc}x(x-y)(x-z) =\sum_{cyc} a^4(a-b)(a-c)-5 \prod_{cyc}(a-b)^2$$

In [66]:
from src.utils.text_process import next_permute
# rep = '(a-b+c)2+(b-c)2'
# rep = '(a2+b2-ab+bc)+2(b2+c2-bc+ca)'
rep = 'a2+(b2+c2)+0.81649658092772603273a(b+c)-4bc'
poly = 's(x(x-y)(x-z))'.replace('x','(%s)'%rep).replace('y','(%s)'%next_permute(rep)).replace('z','(%s)'%next_permute(next_permute(rep)))
poly2 = pl(poly)
print(poly_get_standard_form(poly2, is_cyc = True))
print(poly)

s(188506428383627/8125753663024a4b2-188506428383627/8125753663024a4bc+188506428383627/8125753663024a4c2-188506428383627/2031438415756a3b3+3893476347541514/43975435637405a3b2c+3893476347541514/43975435637405a3bc2-4724745489393389/43959652150154a2b2c2)
s((a2+(b2+c2)+0.81649658092772603273a(b+c)-4bc)((a2+(b2+c2)+0.81649658092772603273a(b+c)-4bc)-(b2+(c2+a2)+0.81649658092772603273b(c+a)-4ca))((a2+(b2+c2)+0.81649658092772603273a(b+c)-4bc)-(c2+(a2+b2)+0.81649658092772603273c(a+b)-4ab)))


In [38]:
pl('s((a2+b2-ab+bc+u(b2+c2-bc+ca))((a2+b2-ab+bc+u(b2+c2-bc+ca))-(b2+c2-bc+ca+u(c2+a2-ca+ab)))((a2+b2-ab+bc+u(b2+c2-bc+ca))-(c2+a2-ca+ab+u(a2+b2-ab+bc))))').as_poly(a,b,c)

Poly((u**2 + u)*a**6 + (-9*u**2 + 9*u)*a**5*b + (u**3 - 9*u + 1)*a**5*c + (9*u**3 - 6*u**2 - 15*u + 9)*a**4*b**2 + (-9*u**3 + 45*u**2 - 18*u - 9)*a**4*b*c + (-15*u**2 + 21*u)*a**4*c**2 + (-8*u**3 + 27*u**2 - 9*u - 8)*a**3*b**3 + (-17*u**3 - 36*u**2 + 81*u - 17)*a**3*b**2*c + (27*u**3 - 45*u**2 - 36*u + 27)*a**3*b*c**2 + (-8*u**3 + 27*u**2 - 9*u - 8)*a**3*c**3 + (-15*u**2 + 21*u)*a**2*b**4 + (27*u**3 - 45*u**2 - 36*u + 27)*a**2*b**3*c + (-9*u**3 + 114*u**2 - 75*u - 9)*a**2*b**2*c**2 + (-17*u**3 - 36*u**2 + 81*u - 17)*a**2*b*c**3 + (9*u**3 - 6*u**2 - 15*u + 9)*a**2*c**4 + (u**3 - 9*u + 1)*a*b**5 + (-9*u**3 + 45*u**2 - 18*u - 9)*a*b**4*c + (-17*u**3 - 36*u**2 + 81*u - 17)*a*b**3*c**2 + (27*u**3 - 45*u**2 - 36*u + 27)*a*b**2*c**3 + (-9*u**3 + 45*u**2 - 18*u - 9)*a*b*c**4 + (-9*u**2 + 9*u)*a*c**5 + (u**2 + u)*b**6 + (-9*u**2 + 9*u)*b**5*c + (9*u**3 - 6*u**2 - 15*u + 9)*b**4*c**2 + (-8*u**3 + 27*u**2 - 9*u - 8)*b**3*c**3 + (-15*u**2 + 21*u)*b**2*c**4 + (u**3 - 9*u + 1)*b*c**5 + (u**2 + u)*c*

In [7]:
rep = 'a2+ub2+vc2+xab+ybc+zca'
poly = 's(i(i-j)(i-k))'.replace('i','(%s)'%rep).replace('j','(%s)'%next_permute(rep)).replace('k','(%s)'%next_permute(next_permute(rep)))
poly = pl(poly).as_poly(a,b,c)

In [37]:
# poly.subs({v:u, z:x}).as_poly(a,b,c)(1,1,sp.sqrt(6)).factor(extension = sp.sqrt(6))
# poly.subs({v:u, z:x}).as_poly(a,b,c).subs(x, (u/6+y/12+sp.S(1)/2)*sp.sqrt(6)).as_poly(a,b,c)
eq1 = poly.subs({c:0,b:1,v:u, z:x}).subs(a,sp.sqrt(3)+2).subs(x, (u/6+y/12+sp.S(1)/2)*sp.sqrt(6)).as_poly(u,y).as_expr().factor(extension = [sp.sqrt(3),sp.sqrt(2)]).args[-1]
# eq2 = poly.subs({c:0,b:1,v:u, z:x}).subs(a,-sp.sqrt(3)+2).subs(x, (u/6+y/12+sp.S(1)/2)*sp.sqrt(6)).as_poly(u,y).as_expr().factor(extension = [sp.sqrt(3),sp.sqrt(2)]).args[-1]

In [170]:
x_ = sp.solve(monom_of(poly,(5,1,0)).as_poly(x), x)[0].factor()
# sp.solve(monom_of(poly,(5,1,0)).as_poly(x), x)[0].factor()
y_ = sp.solve(monom_of(poly,(1,5,0)).as_poly(x).as_expr().subs(x,x_).factor().args[-1].as_poly(y),y)[0].factor()

In [187]:
# monom_of(poly,(6,0,0)).subs(u,0).factor()
eq0 = pl('a2+ub2+vc2+xab+ybc+zca-(b2+uc2+va2+xbc+yca+zab)').subs(v,1).subs(u,1)
eq1 = eq0.subs(a,2).subs(b,sp.sqrt(2)).subs(c,1)
eq2 = eq0.subs(b,2).subs(c,sp.sqrt(2)).subs(a,1)
eq3 = eq0.subs(c,2).subs(a,sp.sqrt(2)).subs(b,1)
sp.solve((eq1,eq2,eq3))

{x: z, y: z}

In [10]:
# (monom_of(poly,(6,0,0)).subs(u,x).subs(v,y))
# poly.as_expr().subs(z,y).subs(u,1).subs(v,1).as_poly(a,b,c)
# pl('s((x(a2+b2-ab+bc)+(b2+c2-bc+ca))((x(a2+b2-ab+bc)+(b2+c2-bc+ca))-(x(b2+c2-bc+ca)+(c2+a2-ca+ab)))((x(a2+b2-ab+bc)+(b2+c2-bc+ca))-(x(c2+a2-ca+ab)+(a2+b2-ab+bc))))').as_poly(a,b,c)

In [15]:
sp.solve(((2*x*x+x*z-2*z*z+1),(2*x*x-4*x*z+2*z*z+1)), (x,z))

[(-2*sqrt(2)*I, -5*sqrt(2)*I/2),
 (-sqrt(2)*I/2, 0),
 (sqrt(2)*I/2, 0),
 (2*sqrt(2)*I, 5*sqrt(2)*I/2)]